<img align="right" src="images/tf-small.png" width="90"/>
<img align="right" src="images/etcbc.png" width="100"/>


# Creating a Coreference-annotated Corpus for Biblical Hebrew

#### An Analysis of Inter-annotator Agreement for Coreference Resolution Annotations in the Psalms and Beyond

## 1. Introduction

This notebook demonstrates and analyses the calculations of inter-annotator agreement, abbreviated to IAA, for the annotation of coreference information in the Hebrew Bible, specifically the Psalms. The Psalms, consisting of 150 poems in Ancient Hebrew, have been chosen as annotation corpus because they are the focus of my PhD research: "Who is who in the Psalms?" 

The Psalms and some comparison texts from Genesis, Numbers and Isaiah, have been annotated according to certain rules: an annotation scheme. A brief explanation of what data is annotated, the process and accompanying annotation tools and resources can be found in the [coreference annotation](https://github.com/cmerwich/participant-analysis/tree/master/annotation) notebooks. Why comparison texts have been annotated is explained below.

A vital part of the annotation process is checking the annotations for inter-annotator agreement. A website on [corpus linguistics](https://corpuslinguisticmethods.wordpress.com/2014/01/15/what-is-inter-annotator-agreement) puts it nicely: 

> "Inter-annotator agreement is a measure of how well two (or more) annotators can make the same annotation decision for a certain category."

From an IAA measure at least three things can be derived: 

* the *reliability* of the annotation scheme or guidelines that describe the category that is being annotated. Do the annotators understand the scheme and can they apply it *independently* and *consistently*? 
* the *reliability* of the annotation process, which is a necessary condition for 
* the *correctness* of the resulting annotations. 

The goal is create a coreference annotation method for the Hebrew Bible that can be used, adapted and corrected by others. In order to do that, we need to know to what extent the annotated information is correct. Producing reliable, consistent and correct coreference annotations are key to developing solid computer-assisted analyses of participants in the Psalms and other Hebrew Bible books. 


### 1.1 Setting up IAA

The scope and financial means of the PhD project do not allow for setting up a large scale IAA process with a team of annotators ranging in level of expertise. Two annotators were available for the current project. In the calculations I therefore function as annotator **A** and a fellow PhD candidate, Gyusang Jin, as annotator **B**. 

10 texts out of 150 Psalms were chosen randomly for annotator **B** with Python's `random` module since the whole Psalms corpus was already annotated by **A**:

```ruby
import random 
for i in range(1,11):
    print(random.randint(1, 151))
```

The output was 11, 88, 101, 17, 70, 138, 20, 32, 67, 129. After a training in the annotation guidelines, **B** started annotating. The 10 resulting annotation files of **B** together with **A**'s annotation files form the input for the IAA algorithm. The annotation files are found under [A](chris_A) and [B](gyus_B). 

The Psalms corpus consists of poetry that is written in Biblical Hebrew. The annotator therefore does not only have to deal with difficulties when trying to understand texts written in a different genre and language, they have also originated in a different space and time. A question that comes up is if it is harder to annotate poetic texts for coreference compared to for example narrative texts.

To enable some comparison between the annotation of different genres Numbers 8-10, which are narrative texts, were also annotated for coreference by **A** and **B**. The Numbers annotations will be analysed at the end of this notebook. The hypothesis for the comparison between coreference annotations in poetry and narrative texts is that annotators **A** and **B** will agree more on their annotations of the narrative texts than on the poetic texts. We will see if this indeed the case. 


### 1.2 Some Theory and the IAA Algorithm

There are all kinds of algorithms for IAA measures. [NLTK](https://www.nltk.org/api/nltk.metrics.html) has implemented some of these agreement metrics. [Artstein and Poesio](https://dl.acm.org/citation.cfm?id=1479206) wrote an informative article on the mathematics of IAA in 2008. I have decided to implement my own algorithm that uses SciPy's [linear sum assignment module](https://docs.scipy.org/doc/scipy-0.18.1/reference/generated/scipy.optimize.linear_sum_assignment.html). This gives me more control over the algorithm's output. 

Calculating IAA for coreference annotations in the Hebrew Bible is cast as an assignment problem. A classic assignment problem is e.g. to assign jobs *{p, q, r, s}* to workers *{a, b, c, d}* so as to minimize the total cost. [Here](http://csclab.murraystate.edu/~bob.pilgrim/445/munkres.html) you can find a nice explanation of this 'Kuhn-Munkres algorithm'. [Wikipedia](https://en.wikipedia.org/wiki/Hungarian_algorithm) is also helpful. 

The implementation of IAA for our case is as follows. Coreference resolution is the task of finding all expressions that refer to the same entity in a text. 

* A referring expression is called a **mention**. 
* An entity can be called a **class**, or **C**. A **class** is a set that contains two or more mentions that refer to the same entity. 
* A mention that refers to an entity that no other mention refers to is called a **singleton**. A **singleton** can be a set that contains one **mention**. I define a **singleton** set **S** that contains all singletons from one text. 

Comparing annotations of `annotator A` and `annotator B` then is done with set calculations. Given the Venn diagram (Source: [wikipedia](https://en.wikipedia.org/wiki/Set_theory)) below, for IAA we are interested in:

<img align="center" src="images/venn_a_intersect_b.png" width="220"/>

* $A \setminus B$ = Left, or blue
* $A \cap B$ = Middle, or purple
* $B \setminus A$ = Right, or pink
* $A \Delta B$ = Symmetric difference of A and B
* $d_{j}(A, B)$ = [Jaccard distance](https://en.wikipedia.org/wiki/Jaccard_index) of A and B

To illustrate, in the following example

| $\setminus$ | A  | $\cap$ | B | $\setminus$ |
| ---- | ---- | ---- | ---- | ---- |
| 0 | C1 | 4 | C1 | 1 |  
| 1 | C2 | 7 | C2 | 1 |
| 8 | C3 | 0 | C3 | 0 |
| 3 | S  | 17 | S  | 2 |  


annotator **A** and annotator **B** have 4 mentions in common in *C1*, **B** has 1 mention more in *C1* than **A**. **A** and **B** have 7 mentions in common in *C2*. Both have annotated 1 mention that they not have in common. For *S*, **A** and **B** have an intersection of 17. For **A** the relative complement, or difference, is 3; for **B** it's 2. 

It is also possible that **A** or **B** has formed a class the other has not: the *C3* class is an example of that.  

The symmetric difference is the calculation of the annotations that belong to **A** or **B** but not to their intersection (Middle). Symmetric difference is defined for sets **A**, **B** as

$$A \Delta B = (A \setminus B) \cup (B \setminus A)$$

So the symmetric distance between **A** and **B** for *C1* is $0+1 = 1$. $A \Delta B$ for *S* is 5. 

The notation for the $d_{j}$ (distance) of a set is

$$ d_{j} = A \Delta B / A \cup B $$

The distance for *S* for example is: $(3+2)/(3+17+2) \approx 0.227 $.

To return to the assignment problem, the annotations of **A** and **B** can be divided as respectively $n$ and $k$ nodes into two disjoint and independent sets $U$ and $V$ in an acyclic bipartite graph (based on: [wikipedia](https://en.wikipedia.org/wiki/Bipartite_graph)). In this bipartite graph every edge connects a node in $U$ to one in $V$:

<img align="center" src="images/bipartite.jpg" width="220"/>

In the [IAA algorithm](iaa.py) then, **A**'s and **B**'s (or $U$ and $V$) sets of annotations of coreference classes and singletons are matched with function `match()` using a distance function `distance()` that calculates the Jaccard distance between the sets. The cost of the matching is calculated with the aforementioned `linear_sum_assignment` SciPy module. The results of the matching are stored in array $r$. The rest of the functions are helper functions that print the calculations to plain text files with extension `.iaa`. These `.iaa` files are found [here](iaa-files).

### 2. Executing the Code

In [1]:
import os
import pandas as pd
from utils import MakeTable, ExportToLatex
from acc import print_total, print_total_corpus
from retrieve_iaa import retrieve_ann
OUTPUT = os.path.expanduser('~/Documents/PhD/1-dissertation/DISSERTATIONlatex/Tables/')

To enable researchers to develop their own annotation process, with different annotators working in different file locations, I have decided to work with a [`Makefile`](Makefile). A `Makefile` does all of the file handling in the terminal, it prevents dependency hell when the annotators decide to change their files after the IAA calculations have been done. With the command `make` the calculations are easily done again. Some instructions:

* make an "iaa"-folder (or give it some other name) somewhere on your computer and place the `iaa.py`, `acc.py` and `Makefile` in that same folder;
* change the file locations for the `.ann` files under NU_A, PS_A etc. in the `Makefile`. Make sure there are **A** and **B** locations for the files of the two different annotators; 
* go to the "iaa"-folder in your terminal and give the command `make`;
* `iaa.py` will do its work and the IAA measures are printed per Hebrew Bible book in separate txt files and it prints one total IAA measure for all compared texts. All files are stored in the "iaa"-folder. 

Jupyter notebooks allow lots of cool magic, another possibility is to just run a shell command in this notebook with `!`, do: 

```ruby
! make
```

In [2]:
! sort -k 7n,7 iaa-files/total_numbers iaa-files/total_psalms

Psalms_138.iaa	-	9	62	10	19	0.2346
Psalms_088.iaa	-	25	121	25	50	0.2924
Psalms_011.iaa	-	12	49	12	24	0.3288
Psalms_129.iaa	-	9	36	9	18	0.3333
Psalms_070.iaa	-	11	34	10	21	0.3818
Psalms_032.iaa	-	21	71	26	47	0.3983
Psalms_020.iaa	-	18	55	19	37	0.4022
Psalms_017.iaa	-	38	107	42	80	0.4278
Psalms_101.iaa	-	19	45	20	39	0.4643
Psalms_067.iaa	-	20	42	21	41	0.494
Numbers_009.iaa	-	79	156	80	159	0.5048
Numbers_008.iaa	-	82	162	86	168	0.5091
Numbers_010.iaa	-	98	174	100	198	0.5323


Let's put all the values per text and the total in a pandas dataframe in which the '-' is dropped and again sort the values in the seventh column ($d_{j}$) by ascending order:

In [3]:
name_ps, Lt_ps, Mt_ps, Rt_ps, Dt_ps, dt_ps = print_total('iaa-files/total_psalms')
name_nu, Lt_nu, Mt_nu, Rt_nu, Dt_nu, dt_nu = print_total('iaa-files/total_numbers')

iaa-files/total_psalms	-	182	622	194	376	0.3768
iaa-files/total_numbers	-	259	492	266	525	0.5162


In [4]:
tot_column_names=['-','L', 'M', 'R', 'D', 'd']
tot_data_types={'-': str, 'L': int, 'M': int, 'R': int, 'D': int, 'd': float}
cols=['L', 'M', 'R', 'D', 'd']

ps_df = pd.read_table('iaa-files/total_psalms', 
                           delim_whitespace=True, 
                           names=tot_column_names,
                           dtype=tot_data_types
                          ).drop(columns='-').sort_values(by='d')

df_ps = pd.DataFrame([[Lt_ps, Mt_ps, Rt_ps, Dt_ps, dt_ps]],
                  index=['total_psalms'],
                  columns=cols
                 )


nu_df = pd.read_table('iaa-files/total_numbers', 
                           delim_whitespace=True, 
                           names=tot_column_names,
                           dtype=tot_data_types
                          ).drop(columns='-').sort_values(by='d')

df_nu = pd.DataFrame([[Lt_nu, Mt_nu, Rt_nu, Dt_nu, dt_nu]],
                  index=['total_numbers'],
                  columns=cols
                 )

total_corpus, L_all, M_all, R_all, D_all, d_all = print_total_corpus()
corpus_df = pd.DataFrame([[L_all, M_all, R_all, D_all, d_all]],
                  index=[total_corpus],
                  columns=cols
                 )

corpus_df
tot_ps_nu_df = ps_df.append(nu_df)

tot_ps_nu = tot_ps_nu_df.append(df_ps)
tot_df = tot_ps_nu.append(df_nu)
all_df = tot_df.append(corpus_df)
all_df

total_corpus	-	441	1114	460	901	0.4471


,L,M,R,D,d
Psalms_138.iaa,9,62,10,19,0.2346
Psalms_088.iaa,25,121,25,50,0.2924
Psalms_011.iaa,12,49,12,24,0.3288
Psalms_129.iaa,9,36,9,18,0.3333
Psalms_070.iaa,11,34,10,21,0.3818
Psalms_032.iaa,21,71,26,47,0.3983
Psalms_020.iaa,18,55,19,37,0.4022
Psalms_017.iaa,38,107,42,80,0.4278
Psalms_101.iaa,19,45,20,39,0.4643
Psalms_067.iaa,20,42,21,41,0.4940


In [5]:
#print(all_df.to_latex(index=True))

ExportToLatex(OUTPUT, 'iaa_all', all_df, indx = True)

### 3. IAA analysis
The calculated IAA measures express a certain degree of disagreement between annotators **A** and **B**. Since the IAA measures do not speak for themselves, they need to be interpreted. Questions that are important for the interpretation of the IAA measures are: How come the IAA measure of this text is are so high? How come **A** and **B** disagree on a certain class matching? What are the differences between their annotations? To give the interpretation and IAA analyis structure, the IAA measures are analysed according to a categorisation of various disagreement types. The categorisation used here is an adapted version of the description from the [OntoNotes CoNNL 2012 shared task](https://dl.acm.org/citation.cfm?id=2391181.2391183), p. 8. 

* *Annotator Error*: an annotator error. This is a catch-all category for errors that do not fit in the other categories. 
* *Genuine Ambiguity*: genuine ambiguous cases where annotators interpret the referent(s) differently.  
* *Generics*: one annotator considers a mention generic and the other annotator does not. This a hard category, since the context of Hebrew Bible texts is far away in time and often unclear. 
* *Guidelines*: the guidelines should be more clear on this example. 
* *Referents*: each annotator thought this was referring to two completely different things. 
* *NP*: one annotator did not mark this nominal phrase. 
* *Suffix*: one annotator did not mark this suffix. 
* *Verb*: one annotator did not mark this verb. 
* *Independent Personal Pronoun*: one annotator did not mark this IPP. 
* *Named Entity*: can be person, measurement unit, people, place, demonstrative personal pronoun. One annotator did not mark this NE.  
* *Demonstrative Pronoun*: one annotator did not mark this DP. 
* *Appositive*: one annotator did not mark this appositive. 
* *brat*: nested mentions cannot be corefered in brat. 

These categories could prove insufficient after the analysis. If this is the case they will be adjusted accordingly. 

### 4. Psalms
Let's pull in the IAA measures for `total_psalms.iaa` with a shell command, and sort the seventh column by ascending order.

In [6]:
! sort -k 7n,7 iaa-files/total_psalms

Psalms_138.iaa	-	9	62	10	19	0.2346
Psalms_088.iaa	-	25	121	25	50	0.2924
Psalms_011.iaa	-	12	49	12	24	0.3288
Psalms_129.iaa	-	9	36	9	18	0.3333
Psalms_070.iaa	-	11	34	10	21	0.3818
Psalms_032.iaa	-	21	71	26	47	0.3983
Psalms_020.iaa	-	18	55	19	37	0.4022
Psalms_017.iaa	-	38	107	42	80	0.4278
Psalms_101.iaa	-	19	45	20	39	0.4643
Psalms_067.iaa	-	20	42	21	41	0.494


Also accumulate the IAA measures of all texts `total_psalms.iaa` with `acc.py`. 

`Lt` stands for Left total, `Mt` for Middle total etc. 

In [7]:
name_ps, Lt_ps, Mt_ps, Rt_ps, Dt_ps, dt_ps = print_total('iaa-files/total_psalms')

iaa-files/total_psalms	-	182	622	194	376	0.3768


The IAA $ d_{j} $ of all Psalms annotated by **A** and **B** is $ 0.3768 $. We'll get back to that value in a bit. 

In [8]:
ps_df = ps_df.append(df_ps)
ps_df

,L,M,R,D,d
Psalms_138.iaa,9,62,10,19,0.2346
Psalms_088.iaa,25,121,25,50,0.2924
Psalms_011.iaa,12,49,12,24,0.3288
Psalms_129.iaa,9,36,9,18,0.3333
Psalms_070.iaa,11,34,10,21,0.3818
Psalms_032.iaa,21,71,26,47,0.3983
Psalms_020.iaa,18,55,19,37,0.4022
Psalms_017.iaa,38,107,42,80,0.4278
Psalms_101.iaa,19,45,20,39,0.4643
Psalms_067.iaa,20,42,21,41,0.4940


The psalms dataframe (`ps_df`) is built up of the IAA calculations per text. The colums are named L(eft) and R(ight) for the relative complement, M(iddle) for intersection, the symmetric difference is D for $\Delta$, and d for $d_{j}$. 

The $d_{j}$ is a value $0 \leq d_{j} \leq 1 $ where $0$ denotes total inter-annotator agreement and $1$ total inter-annotator *dis*agreement. Considering the values of $d_{j}$ in the `total_psalms` dataframe the question arises what $d_{j}$ value threshold is maintained to be able to speak of IAA. Or in other words: here we encounter the problem of interpreting the meaning of the resulting values. To quote Artstein and Poesio:

> "Unfortunately, deciding what counts as an adequate level of agreement for a specific purpose is still little more than a black art: [a]s we will see, different levels of agreement may be appropriate for resource building and for more linguistic purposes." 

[Artstein and Poesio, 2008](https://dl.acm.org/citation.cfm?id=1479206), p.576

To put it in another away, though IAA is important, it is hard to analyse. This statement nevertheless does not withhold them from concluding that in relation to [Krippendorff's](https://en.wikipedia.org/wiki/Krippendorff%27s_alpha) $\alpha$ - an agreement coefficent for multiple coders where 1 signifies perfect agreement:

> "only values above 0.8 ensured an annotation of reasonable quality. We therefore feel that if a threshold needs to be set, 0.8 is a good value." 

[Artstein and Poesio, 2008](https://dl.acm.org/citation.cfm?id=1479206), p.591

Taking into account some of the difficulties that the corpus poses however - see the remarks under §1.1 'Setting up IAA' - I suggest we start doing some black art ourselves. I set a 'study' IAA threshold for individual texts at IAA $\leq 0.333$ with which the texts with a low IAA measure can be selected for annotation analysis. 

In [9]:
ps_df.loc[(ps_df['d'] >= 1/3)]

,L,M,R,D,d
Psalms_070.iaa,11,34,10,21,0.3818
Psalms_032.iaa,21,71,26,47,0.3983
Psalms_020.iaa,18,55,19,37,0.4022
Psalms_017.iaa,38,107,42,80,0.4278
Psalms_101.iaa,19,45,20,39,0.4643
Psalms_067.iaa,20,42,21,41,0.4940
total_psalms,182,622,194,376,0.3768


In [10]:
ps138_df = MakeTable('iaa-files/Psalms_138.iaa')

ps138_df

,ann_A,ann_B,L,M,R,D,d
0,C1,C1,0,14,1,1,0.0667
1,C2,C3,1,23,4,5,0.1786
2,C3,C2,0,4,0,0,0.0000
3,C4,C4,0,2,0,0,0.0000
4,S,S,2,19,5,7,0.2692
5,C5,-,2,0,0,2,1.0000
6,C6,-,4,0,0,4,1.0000


### 4.1 Psalms_067.iaa 

Starting with `Psalms_067.iaa` with $d_{j} \approx 0.4940$, the file can be loaded in a pandas dataframe for clarity. We use a function `MakeTable()` in which the iaa file location can be passed as argument to make the pandas dataframe. The column names stay the same: `ann_A`,`ann_B`, `L`, `M`, `R`, `D` and `d`: 

In [11]:
ps067_df = MakeTable('iaa-files/Psalms_067.iaa')

ps067_df

,ann_A,ann_B,L,M,R,D,d
0,C1,C5,0,6,0,0,0.0000
1,C2,C1,5,0,9,14,1.0000
2,C3,C3,2,3,0,2,0.4000
3,C4,C2,1,12,7,8,0.4000
4,C5,C4,9,12,0,9,0.4286
5,C6,C6,0,2,0,0,0.0000
6,S,S,1,7,5,6,0.4615
7,C7,-,2,0,0,2,1.0000


Recall that the IAA algorithm matches the *C* and *S* sets of annotations from **A** and **B** in the most optimal way. That is why different class numbers can be matched differently for **A** and **B**. The different matching of class numbers does not mean that they have been identified as different entities. For indices 0 and 5

In [12]:
ps067_df.loc[[0,5]]

,ann_A,ann_B,L,M,R,D,d
0,C1,C5,0,6,0,0,0.0
5,C6,C6,0,2,0,0,0.0


means that there is complete agreement between **A** and **B** on which mentions refer to a certain entity. Indices 1 and 7

In [13]:
ps067_df.loc[[1,7]]

,ann_A,ann_B,L,M,R,D,d
1,C2,C1,5,0,9,14,1.0
7,C7,-,2,0,0,2,1.0


indicate that there is complete disagreement between **A** and **B** on which mentions refer to a certain entity.

* *C2* and *C1* are matched, but **A** and **B** have found that different mentions refer to that entity. 
* **A** has found an extra class, which can be concluded from both the non-match (i.e. '-') and the extra class that **A** (i.e. *C7* or *C2*) has found compared to **B**. 

For the sets with $d_{j} \approx 0.4 $:

In [14]:
ps067_df.loc[[2,3,4,6]]

,ann_A,ann_B,L,M,R,D,d
2,C3,C3,2,3,0,2,0.4000
3,C4,C2,1,12,7,8,0.4000
4,C5,C4,9,12,0,9,0.4286
6,S,S,1,7,5,6,0.4615


* on index 3, **B** connects 7 mentions to his *C2*
* on index 4, while **A** connects 9 mentions to his *C5*, **B** connects 0 mentions to his *C4*.
* in *S* **A** and **B** have 7 mentions in common, but **B** finds 5 more mentions as singletons.  

Now that we have a general overview of what the differences in classes are, let's take a closer look at the specific differences in the mentions that have been included in each class by each annotator. Since the optimal matching between the **A**'s and **B**'s annotations are always the same, unless changes have been made in the annotations, it is possible to retrieve the actual words of the annotations. In the cell below the function `retrieve_ann`, imported from `retrieve_iaa.py`, runs the comparison again and prints per class matching:

1. on the first line, between hyphens, the comparison of a class with IAA measures;
2. on the second line the annotations of **A** in ETCBC's [transcription](https://annotation.github.io/text-fabric/Writing/Hebrew.html) of Biblical Hebrew. The annotations have also been done on the transcribed texts. `un A` indicates unpaired classes of **A** that have not been matched with **B**;
3. on the third line the annotations of **B** in ETCBC's transcription. `un B` indicates unpaired classes of **B** that have not been matched with **A**. 
4. on the fourth and fifth lines, the specific differences (i.e. `diff`) in annnotations between **A** and **B** if they exist. If there are no differences these lines will not be printed. The notation is: the annotator, **A** or **B**, followed by `diff`, then by a word in brackets together with an index number that points to the word's position in **A** or **B**'s class. An example:

`A	['JSWR', 'LBB']`

`B	['JSWR', 'LBB <QC']`

`A diff	[('LBB', 1)]`	

`B diff	[('LBB <QC', 1)]`

Meaning: **A** has one annotation that **B** does not have, and the other way around. **A**'s differing annotation can be found on index 1 (all indices start at 0), in **A**'s annotations above. **B**'s differing annotation can also be found on index 1 in **B**'s annotations above. If **A** has differing annotations that **B** does not have, or vice versa, an empty list is shown like so:

`A diff	[('>RY', 0), ('>RY', 1)]`	

`B diff	[]`

To make sense of the transcripted annotations in Hebrew, and trace which Hebrew word where in the text corresponds to which annotation I make use of the `2.annotation_aid` in the [coreference annotation](https://github.com/cmerwich/participant-analysis/tree/master/annotation) GitHub folder. I run the notebook and under the header '5.1 Choose book and chapter' I fill in the Bible book (Psalms) and the chapter(s) I want to study. 

In [15]:
retrieve_ann('chris_A/Psalms_067.ann', 'gyus_B/Psalms_067.ann')

ann_A	ann_B	L	M	R	D	d	

------------------------------------------------------------
C1	C5	0	6	0	0	0.0
------------------------------------------------------------
A	['NW', 'NW', 'NW', 'NW', 'NW', 'NW']	

B	['NW', 'NW', 'NW', 'NW', 'NW', 'NW']	

------------------------------------------------------------
C2	C1	5	0	9	14	1.0
------------------------------------------------------------
A	['JFMXW', 'JRNNW', 'L>MJM', 'L>MJM', 'M']	

B	['>LHJM', 'JXN', 'JBRK', 'J>R', 'W', 'K', 'K', 'TCPV', 'TN']	

A diff	[('JFMXW', 0), ('JRNNW', 1), ('L>MJM', 2), ('L>MJM', 3), ('M', 4)]	

B diff	[('>LHJM', 0), ('JXN', 1), ('JBRK', 2), ('J>R', 3), ('W', 4), ('K', 5), ('K', 6), ('TCPV', 7), ('TN', 8)]	

------------------------------------------------------------
C3	C3	2	3	0	2	0.4
------------------------------------------------------------
A	['>RY', '>RY', '>RY', 'NTNH', 'H']	

B	['>RY', 'NTNH', 'H']	

A diff	[('>RY', 0), ('>RY', 1)]	

B diff	[]	

------------------------------------------------------------


1. C2 - C1 $d_{j} = 1$ || C4 - C2 $d_{j} = 0.4$:

    * **B**'s first '>LHJM' class is matched with **A**'s *C2* ('L>MJM'). 
    * **B**'s class *C4* contains mentions that also refer to '>LHJM'. Since **B**'s *C4* contains '>LHJM', it is probably a linking error made in brat. **B**'s diff for his *C1* matches **A**'s diff for his *C5*. The differences (2x 9) can therefore be added up and classified as 18x _Annotator Error_.
    * 'XM' added by **B** indicates in *C2* is 1x _Annotator Error_. In v. 5 the verb 'TNX' occurs with suffix M. 'TNX' refers to '>LHJM', 'M' to '<MJM'. The mentions were not annotated correctly.
    * 'WDW' added by **B** indicates in *C2* is 1x _Annotator Error_. In v. 6 the verb 'JWDW'occurs. **B** forgot to annotate the letter _yod_. **A**'s diff is also counted as 1x _Annotator Error_.
    * The rest of the differences in mentions added to the **A** and **B** classes in *C4* - *C2* is the result of an interpretation issue, thus categorised as 5x _Genuine Ambiguity_. The question is if '<MJM', 'L>MJM' and 'KL&GWJM' all refer to the same 'peoples'/'nations' or not. This interpretation question also causes the difference in the number of classes and the number of singletons. Where **A** sees one class of '<MJM' (*C4*), one class of 'L>MJM' (*C2*), and a singleton in KL&GWJM; **B** adds all these mentions in one class (*C2*). The difference of **A**'s *C2* and the singleton is counted as 6x _Genuine Ambiguity_. 
    
    -----------------------------------------------------------

2. C3 - C3 $d_{j} = 0.4$:

    * **A** added two mentions of '>RY' more than **B**.
    * One occurrence of '>RY' in **A**'s class is 1x _Annotator Error_, it occurs in v.7 and is undetermined. It refers to 'land' that yields harvest, and does not refer to 'the earth' as with the other mentions. The difference this error causes in the singletons match is also counted as 1x _Annotator Error_. 
    * **B** missed a determined mention of '>RY': 1x _Annotator Error_. Therefore one mention ended up as singleton in **B**'s *S* set which it shouldn't have: 1x _Annotator Error_. 
    
    -----------------------------------------------------------
    
4. C5 - C4 $d_{j} \approx 0.4286$ || C2 - C1 $d_{j} = 1.0$:

    * Compared to **B**, **A**'s class has a greater number of mentions, the mentions refer to '>LHJM', God. **B**'s class *C1* contains mentions that also refer to '>LHJM'. Since **B**'s *C1* contains '>LHJM', it is probably a linking error made in brat. **B**'s diff for his *C1* matches the **A**'s diff for his *C5*. The differences (2x 9) can therefore be added up and classified as 18x _Annotator Error_.
    
    -----------------------------------------------------------

5. S - S $d_{j} \approx 0.4615$ || unpaired *C7*: 

    * **A**'s unpaired *C7* contains the title of Psalm 67. The annotation guidelines prescribe that the titles of Psalms are annotated as self-contained units, and that they are not connected to coreference chains in the text that follows. While both **A** and **B** have followed that guideline, **B** has missed the apposition relation between 'CJR' and 'MZMWR'. According to the guidelines this relation should be annotated, this difference is thus categorised as 4x _Appositive Error_ (also 2x singletons). 
    * The singleton that has not been discussed yet is **B**'s ('D<T', 5). 'D<T' is an infc without object or subject suffix and has no lexical identity: 1x _Annotator Error_.

The matching of the remaining classes have already been discussed in relation to aforementioned classes. 


#### Conclusion Psalm 67
The relatively low IAA measure can mostly be attributed to 

* 44x _Annotator Error_ and 
* 11x _Genuine Ambiguity_ cases and 
* 4x _Appositive_ error. 

The _Annotator Error_ s and _Appositive_ error can easily be corrected, the _Genuine Ambiguity_ has to be left for what it is.

### 4.2 Psalms_101.iaa 

`Psalms_101.iaa` has the second most low IAA measure with $d_{j} \approx 0.4643$. Following the same analysis procedure as with `Psalms_064.iaa`, we first make and inspect a table of the IAA measures per class. Then we retrieve the actual words of the annotations to take a closer look at the differences in annotations. 

In [16]:
ps101_df = MakeTable('iaa-files/Psalms_101.iaa')

ps101_df

,ann_A,ann_B,L,M,R,D,d
0,C1,C1,0,3,0,0,0.0000
1,C2,C2,1,20,1,2,0.0909
2,C3,C4,1,1,1,2,0.6667
3,C4,C5,2,1,1,3,0.7500
4,C5,C3,2,0,2,4,1.0000
5,C8,C6,1,2,0,1,0.3333
6,C10,C7,0,2,0,0,0.0000
7,C11,C8,0,2,0,0,0.0000
8,S,S,6,14,15,21,0.6000
9,C9,-,2,0,0,2,1.0000


The dataframe `ps101_df` immediately shows some striking differences in annotations. **A** and **B** agree (almost) completely on four classes (index 0, 1, 6, 7) with $d_{j}$ being approximately 0. For the remaining classes there is a disagreement of .3333 or greater. We leave the classes that are absolutely similar out of consideration and we take a look at the rest of the classes. The remaining classes are selected by sorting all $d_{j}$'s that are equal or greater than 0.01 and we retrieve all annotations:

In [17]:
ps101_df.loc[(ps101_df['d'] >= 0.01)]

,ann_A,ann_B,L,M,R,D,d
1,C2,C2,1,20,1,2,0.0909
2,C3,C4,1,1,1,2,0.6667
3,C4,C5,2,1,1,3,0.7500
4,C5,C3,2,0,2,4,1.0000
5,C8,C6,1,2,0,1,0.3333
8,S,S,6,14,15,21,0.6000
9,C9,-,2,0,0,2,1.0000
10,C6,-,2,0,0,2,1.0000
11,C7,-,2,0,0,2,1.0000


In [18]:
retrieve_ann('chris_A/Psalms_101.ann', 'gyus_B/Psalms_101.ann')

ann_A	ann_B	L	M	R	D	d	

------------------------------------------------------------
C1	C1	0	3	0	0	0.0
------------------------------------------------------------
A	['K', 'JHWH', 'TBW>']	

B	['K', 'JHWH', 'TBW>']	

------------------------------------------------------------
C2	C2	1	20	1	2	0.0909
------------------------------------------------------------
A	['>CJRH', '>ZMRH', '>FKJLH', 'J', '>THLK', 'J', 'J', '>CJT', 'J', 'FN>TJ', 'J', 'NJ', '>D<', '>YMJT', '>WKL', 'J', 'J', 'NJ', 'J', 'J', '>YMJT']	

B	['>CJRH', '>ZMRH', '>FKJLH', 'J', '>THLK', 'J', 'J', '>CJT', 'J', 'FN>TJ', 'J', 'J', '>D<', '>YMJT', '>WKL', 'J', 'J', 'NJ', 'J', 'J', '>YMJT']	

A diff	[('NJ', 11)]	

B diff	[('J', 11)]	

------------------------------------------------------------
C3	C4	1	1	1	2	0.6667
------------------------------------------------------------
A	['LBB', 'JSWR']	

B	['LBB <QC', 'JSWR']	

A diff	[('LBB', 0)]	

B diff	[('LBB <QC', 0)]	

------------------------------------------------------------
C4	C

Let's start at index 1 in the `ps101_df` table and work our way down to 11. 

1. C2 - C2 $d_{j} \approx 0.0909$:

    * Both **A** and **B** annotated 11 occurrences of a first person singular (1Csg). They differ in their annotations of the different forms of 1Csg: J or NJ. Of the word MIM.EN.IJ, 'from me', in v. 101:4 **A** annotated NJ, where **B** annotated J. According to the `annotation_resources` and the annotation guidelines J is an annotation mistake, since ETCBC's analysed form is (M(N&M&N+(NJ and NJ should be annotated. It is therefore categorised as 1x _Annotator Error_.
    
    -----------------------------------------------------------
    
2. C3 - C4 $d_{j} \approx 0.6667$:

    * Where **A** has annotated 'heart' ('LBB'), **B** has also annotated the adjective: 'crooked heart' ('LBB <QC'). Though the annotation guidelines indeed allow for the annotation of adjectives as mention, they are not coreferenced with other referents by themselves. Conversely, **A** annotates 'DRK' ('way', see point 5 below) without the adjective 'TMJM' ('complete'), but annotates 'TMJM' as a seperate mention, which is incorrect, and adds it to his *S* class. **B** has not annotated 'TMJM' as mention. The annotation guidelines therefore need to be revised. These 2 errors are categorised as 2x _Guidelines_.
    
    -----------------------------------------------------------
    
3. C4 - C5 $d_{j} = 0.75$:

    * In 101:5 **A** incorrectly annotated 'W' instead of 'HW', like **B** did. Both 'W' and 'HW' are suffix forms of 3Msg. The `annotation_resources` and the annotation guidelines indicate for R;<;HW ('his neighbour') that the suffix (R<=/+HW) 'HW' should be annotated. This error is categorised as 2x _Annotator Error_. As a consequence 'R<H' is added to **A**'s *S* class instead of **B**'s correct 'R<': 2x _Annotator Error_. 
    * In the same verse **B**, in contrast to **A**, did not corefer the participle 'MLWCNJ' ('he who slanders') to his class and ended up as singleton in *S*. The same goes for the second 'W' which **A** corefered to this class and **B** did not, making it a singleton. Though 'MLWCNJ' is determined as a participle (Msg) and does not have a person feature that refers to the ensuing 3Msg suffixes, they do corefer. These errors are categorised as 4x _Annotator Error_: 2x for the match, 2x for the singletons. 
    
    -----------------------------------------------------------
    
4. C5 - C3 $d_{j} = 1.0$:

    * In this match **A** corefers 'LBB' ('heart') with 'W' in 101:5. As the suffix 'W' is attached to an object marker ('>T'), the 3Msg refers to 'he who slanders' (cf. point 3 (C4 - C5)), not to 'heart'. This is 2x _Annotator Error_. **B** has not corefered 'W' and 'LBB', hence they are added to his *S* class. **A**'s missing singletons are 2x _Annotator Error_.
    * **B** corefers 'JDBQ' with 'DBR&BLJ<L' in 101:3. The question is if 'JDBQ', (verb, 3Msg, to cling to), and 'DBR&BLJ<L' (NP, 'word of wickedness', both words are Msg) corefer. 'DBR&BLJ<L' occurs in a clause that does not directly precede 'JDBQ'. 'JDBQ' is preceded by the clause 'I hate the doing of (the) faithless', '>FH SVJM 'FN>TJ'. It is therefore more probable that 'JDBQ' corefers as follows: 'the doing of the faithless, it/he does not cling to me'. Both **A** and **B** have clearly interpreted these mentions differently, the differences are therefore categorised as ambiguity. 2x _Genuine Ambiguity_ for **B**'s diff in *C3*, 2x for **A** adding JDBQ' and 'DBR&BLJ<L' to his singletons. Though '>FH' is clearly an infinitive that should not be annotated, it functions together with 'SVJM' as an 'NP': 'the doing of the faithless'. **A** has not annotated '>FH', **B** adds '>FH' to his singletons: 2x _Genuine Ambiguity_. 
    
    -----------------------------------------------------------
    
5. C8 - C6 $d_{j} \approx 0.3333$:

    * In 101:6 **A** corefers the complement 'DRK' with 'HW>' (IPP, 3Msg, 'he') and 'JCRT' (verb, 3Msg, 'to serve'). If **A** were to be followed, 'he' ('HW>') would refer to 'the way' ('DRK'), and the way would 'serve': 'the way, he, he, serves'. This is not correct since 'he' ('HW>') is subject of 'JCRT': 'he serves'. This error is categorised as 2x _Annotator Error_ (1x for the difference and 1x for the singletons).
    
    -----------------------------------------------------------
    
6. S - S $d_{j} = 0.6$:

    * Some of the annotated singletons of **A** and **B** have already been discussed in relation to the matched classes. There are however a few differences that are worth looking into. 
    * **B**'s 'WMCPV' corresponds to **A**'s 'MCPV', but the former has incorrectly annotated the 'W' ('and'): it is therefore 2x _Annotator Error_. 
    * **A** and **B** have annotated three occurrences of '<JN' (substantive, determined by a suffix 1Csg, 'eye') in 101:3, 101:6 and 101:7 differently. **A** correctly corefers two occurrences of the determined substantive making it an unmatched class (*C6*). **A** also misses one occurrence ('<JN', 15, in *S*) hence making it 1x _Annotator Error_. **B** annotates all mentions, 2x ('NGD <JN', 7), 1x ('<JN', 17), but does not corefer them. These are 5x an _Annotator Error_ (3x for the *S* and 2x for **A**'s *C6*). Furthermore **B** annotates 'NGD <JN' in 101:3 and 101:7. Strictly speaking the annotation guidelines allow for the annotation of 'NGD', meaning 'counterpart' or 'before', but this substantive seems to function more as a preposition. The difference in translation is 'the counterpart of my eyes' versus 'before my eyes'. **B** has chosen the former, **A** the latter. These 2 errors are categorised as _Guidelines_. The guidelines should be adjusted. 
    * Whereas **B**'s has added 'QRB BJT' to his *S* class, **A** adds the two occurrences to a separate class (*C9*). 'QRB BJT', 'the interior of the house' occurs in 101:2 and 101:7 and both NP's are determined by a suffix 1Csg. They should therefore be coreferenced in a class as **A** has done. **B** has made 4x _Annotator Error_ (2x for **A**'s *C9* and 2x for **B**'s *S*). 
    * Though **A** has coreferenced one occurrence of 'DRK' to his *C8* class, he has also missed an occurrence that **B** has added to his *S* class. 'DRK' occurs in 101:2 and 101:5. **A** has made 2x an _NP_ error. 
    * Lastly, **A** annotates an additional and unpaired *C7* class with 'N>MNJ&>RY' and 'HLK'. It is however not 'N>MNJ&>RY' ('faithful in the land') that should be corefered with 'HLK' (particple, Msg, 'he who walks'), but 'HW>' and 'JCRT' from the *C8 - C6* match making it: 'he who walks, he, he, serves'. This error is categorised as 2x _Annotator Error_ for **A** and 2x _Annotator Error_ for **B** who has added these mentions to his singletons. 


#### Conclusion Psalm 101

The low IAA measure is a result of:

* 31 _Annotator Error_'s, 
* 4 _Guidelines_ errors and 
* 2x _NP_ error. 

These can all be easily corrected.

### 4.3 Psalms_017.iaa 

`Psalms_017.iaa` has an IAA measure of $d_{j} \approx 0.4278$. Let's see how that measure has come to be. 

In [19]:
ps017_df = MakeTable('iaa-files/Psalms_017.iaa')

ps017_df

,ann_A,ann_B,L,M,R,D,d
0,C1,C11,9,27,0,9,0.2500
1,C2,C1,0,2,0,0,0.0000
2,C3,C2,0,2,0,0,0.0000
3,C4,C3,0,2,0,0,0.0000
4,C5,C4,0,2,0,0,0.0000
5,C6,C5,7,20,2,9,0.3103
6,C7,C9,2,0,3,5,1.0000
7,C8,C8,3,0,5,8,1.0000
8,C9,C7,0,12,3,3,0.2000
9,C10,C10,2,2,0,2,0.5000


Like in the previous analyses we study all classes, or sets, of which $d_{j}$ is not 0. The annotation choice an annnotator makes for one class or mention, has influence on the way other classes are built up. The differences in annotations between the classes therefore need to be studied in relation to each other. For `ps017_df` let's zoom in on the classes with an IAA measure of 0.2 or greater: 

In [20]:
ps017_df.loc[(ps017_df['d'] >= 0.2)]

,ann_A,ann_B,L,M,R,D,d
0,C1,C11,9,27,0,9,0.2500
5,C6,C5,7,20,2,9,0.3103
6,C7,C9,2,0,3,5,1.0000
7,C8,C8,3,0,5,8,1.0000
8,C9,C7,0,12,3,3,0.2000
9,C10,C10,2,2,0,2,0.5000
10,C11,C6,2,0,10,12,1.0000
11,C13,C13,0,2,4,4,0.6667
12,C14,C12,3,2,0,3,0.6000
13,S,S,6,34,15,21,0.3818


In [21]:
retrieve_ann('chris_A/Psalms_017.ann', 'gyus_B/Psalms_017.ann')

ann_A	ann_B	L	M	R	D	d	

------------------------------------------------------------
C1	C11	9	27	0	9	0.25
------------------------------------------------------------
A	['CM<H', 'JHWH', 'HQCJBH', 'H>ZJNH', 'K', 'K', 'BXNT', 'PQDT', 'YRPT', 'TMY>', 'K', 'K', 'K', 'T<N', '>L', 'HV', 'K', 'CM<', 'HPLH', 'K', 'K', 'CMR', 'K', 'TSTJR', 'QWMH', 'JHWH', 'QDMH', 'HKRJ<', 'PLVH', 'K', 'K', 'JHWH', 'K', 'TML>', 'K', 'K']	

B	['CM<H', 'JHWH', 'HQCJBH', 'H>ZJNH', 'K', 'K', 'BXNT', 'PQDT', 'YRPT', 'TMY>', 'K', 'K', 'K', 'T<N', '>L', 'QWMH', 'JHWH', 'QDMH', 'HKRJ<', 'PLVH', 'K', 'K', 'JHWH', 'K', 'TML>', 'K', 'K']	

A diff	[('HV', 15), ('K', 16), ('CM<', 17), ('HPLH', 18), ('K', 19), ('K', 20), ('CMR', 21), ('K', 22), ('TSTJR', 23)]	

B diff	[]	

------------------------------------------------------------
C2	C1	0	2	0	0	0.0
------------------------------------------------------------
A	['MCPV', 'JY>']	

B	['MCPV', 'JY>']	

------------------------------------------------------------
C3	C2	0	2	0	0	0.

1. C1 - C11 $d_{j} = 0.25$ || C11 - C6 $d_{j} = 1.0$:

    * **A** and **B** recognise a 'JHWH' ('God') class in their *C1* and *C11* classes respectively. **A** has added nine mentions more to his class than **B**. These 9 mentions plus 'MWCJ<' are **B**'s *C6* class which is matched with **A**'s *C11*. **A** adds 'MWCJ<' in 17:7 (NP vocative, Msg, 'saviour'/'helper') to his singleton set. The difference of mentions as indicated by **A**'s diff for this *C1* - *C11* match seems to be caused by a linking error made in brat by **B**. All mentions (verbs: 'HV', 'CM<', 'HPLH', 'CMR', 'TSTJR'; suffixes 'K') refer in 2Msg to God. **A** forgot to link 'MWCJ<' to this class: 2x _Annotator Error_ (1x for the diff, 1x for the singletons). **B**'s linking error is counted as 18x _Annotator Error_ (2 x 9 for both diffs, ). Making it 20x an _Annotator Error_.
    * **A** distinguishes a 'NPC' (Fsg, 'soul') class in *C11*. **B** has annotated them as two singletons. The first occurrence of 'NPC' in 17:9 is an undetermined NP, but clearly refers to the soul of the I person that is being threatened by enemies. This can be inferred from the three suffixes 1Csg that occur in this verse. The 'I', the 'NPC' or self is under threat. The second occurence of 'NPC' in 17:13 is a determined NP with a 1Csg suffix. Here the same I person asks God to save him ('my soul') from 'the sword' (i.e. a threat) of the enemy. These two differences are categorised as 4 cases of _Genuine Ambiguity_ (2x diff, 2x singletons).
    
    -----------------------------------------------------------
   
2. C6 - C5 $d_{j} \approx 0.3103$ || C8 - C8 $d_{j} = 1.0$:

    * **A** adds seven mentions to his *C6* class that **B** misses. **B**'s diff indicates that he has added two mentions ('DWD' and  'TJ') that **A** misses. Starting with those, 'DWD' (nomen proprium, 'David') belongs to the title of Ps 17. As was the case with Ps 67, according to the guidelines titles of Psalms are annotated for coreference but are not connected to coreference information in the rest of the Psalm. 'TJ' is supposed to be the suffix 1Csg 'J', it is connected to '>MRT' ('word') in 17:6, but **B** has erroneously also annotated the 'T'. **B**'s 'DWD' and  'TJ' is 5x an _Annotator Error_ (2x for the diff, 3x for the *S*). This also explains two of the differences in **A**'s and **B**'s *S*, namely: 'DWD' and '>MRT'. 
    * For the remaining differences, the seven mentions in **A**'s *C6*, there are two explanations. The first one is that the difference is a result of Genuine Ambiguity. In the *C8* - *C8* match, **B**'s *C8* agrees with the seven mentions in **A**'s *C6* diff, except for '>FB<H' (verb, 1Csg, 'to be sated') and the aforementioned (T)'J' suffix 1Csg. While **B** explictely identifies his *C5*, which contains 1Csg mentions, as 'David'. **A** leaves the 1Csg mentions class unidentified. Possibly, **B** started a new coreference class because of the sudden occurrence of the suffix 'NW' (1Cpl) in 17:11. Strangely, **B** adds '>XZH' (verb, 1Csg, 'to see') in 17:15 to his *C8* class, but leaves '>FB<H' in the same verse out which is then added to *S*. In terms of consistency '>FB<H', 'I am sated', should also have been added to this class. The second explanation - the difference is a result of an _Annotator Error_ - is therefore more probable. **B** did not link all the 1Csg mentions correctly. We count 12x an _Annotator Error_ (6x for **A**'s *C6* diff and **B**'s *C8* diff).
   
    -----------------------------------------------------------

3. C7 - C9 $d_{j} = 1.0$ || and C10 - C10 $d_{j} = 0.5$:

    * These two matchings are discussed in relation to each other. 
    * For 17:7 **A** incorrectly links 'XWSJM' to 'MTQWMMJM' in his *C7*. 'XWSJM' (participle, Mpl, 'to seek refuge'), translated as 'they\those who seek refuge' is not the same referent as 'MTQWMMJM' (participle, Mpl, 'to arise'), 'they/those who arise'. The translation of this part of 17:7 would be: 'Oh you helper [God] for those who seek refuge from those who arise against your [God] right hand'. 'XWSJM' and 'MTQWMMJM' therefore end up in **B**'s *S* class. We categorise and count 4x an _Annotator Error_ (2x for the diff, 2x for the singletons).
    * **A** and **B** differ on their understanding of the lion metaphor in 17:12 with translation: 'His likeness is of a lion, he longs to tear; and like a young lion who sits in a hiding place'. **B** annotates two classes. **B** links 'KPJR' ('young lion') to 'JCB' (participle, Msg, 'to sit') in 17:12 in a separate class *C10*. **A** links these two mentions to '>RJH' ('lion') and 'JKSWP' (verb, 3Msg, to 'to long') in one *C10* class: '>RJH', 'JKSWP', 'KPJR', 'JCB'.  **B** adds 'VRWP' (infinitive cst, 'to tear') to his *C10*. According to the annotation guidelines infinitives are not annotated unless there is lexical identity with the same verb elsewhere. 'To tear' does not occur elsewhere in Ps 17, 'VRWP' is therefore 1x an _Annotator Error_. The question is then: do 'the lion' and 'the young lion' corefer or not, are they the same referents? Though a young lion is still a lion, it is probable in this text that the two lions are not the same. Therefore: 4x a case of _Referents_ (2x for **B**'s *C10* diff, 2x for **A**'s *C9* diff).
   
    ------------------------------------------------------------

4. C9 - C7 $d_{j} = 0.2$ || C8 - C8 $d_{j} = 1.0$:

    * **B**'s *C8* is left out of consideration, since it has already been discussed under point 2, **A** is right in linking 'PNJ RC<JM' and 'CDW' in  17:9 in his *C8*, but incorrectly links 'RC<' from 17:13 to this class. 'PNJ RC<JM' (NP, Mpl, 'faces of the wicked') does not corefer with 'RC<' (NP, Msg, 'a wicked') which has a singular form. 'CDW' (verb, 3Cpl, 'to despoil') corefers with the plural of 'PNJ RC<JM'. This is counted as 2x an _Annotator Error_ (1x for 'RC<', 1x for the diff with **B**'s singletons). 
    * **B** links 'PNJ RC<JM' and 'CDW' together with 'NVWT' (infinitive, 'to extend') to his *C7*. Since infinitives without suffix are not annotated as mention, 'NVWT' is categorised as 1x _Annotator Error_. 
    * Where **A** sees the 'faces of the wicked' as a separate class from the '>JB' (participle, Mpl, 'be hostile'/'enemies) class (*C9*) that starts in 17:9, **B** considers 'PNJ RC<JM' and '>JB' belonging to one class (*C7*). Whether 'the faces of the wicked' and 'the enemies' refer to the same entity is hard to say. Therefore, these differences between **A** and **B** are categorised as 2x _Genuine Ambiguity_, one for **A** and one for **B**.

    ------------------------------------------------------------
 
5. C13 - C13 $d_{j} \approx 0.6667$ || C14 - C12 $d_{j} = 0.6$ || unpaired C12 **A**:

    * The annotation choices made by **A** and **B** in 17:14 are best understood when discussing the three matchings in relation to each other. **A**'s *C12* consists of 2x 'MTJM' (substantive, Mpl, 'man'). **A** distinguishes a separate *C13* class with two suffixes 3Mpl ('M'). **B**'s *C13* connects one occurrence of 'MTJM' to the two 3Mpl suffixes together with 'HNJXW' (verb, 3Cpl, 'to settle'), 'M', and 'HM' (both 3Mpl suffixes). **B** thus identifies the 3Mpl mentions as 'a man/a people' ('MTJM'). **B** correctly connects 'M' and 'M' with 'MTJM', but forgets to connect the second 'MTJM', which is thus added to his singletons. That **A** misses a link between his *C12* and *C13* is 2x _Annotator Error_ (2x 'MTJM'). **B** forgetting the second 'MTJM' is also 2x _Annotator Error_ (1x for the class diff and 1x for *S*).
    * As mentioned **B** adds 'HNJXW', 'M' and 'HM' to his *C13*. **A** however considers 'JFB<W', 'BNJM', 'HNJXW', 'M' and 'HM' as one class (*C14*) of which 'BNJM' (substantive NP, Mpl, 'children') is the subject. **B** sees a separate class in 'JFB<W' and 'BNJM' (*C12*). The subject of 'HNJXW' is, as **A** has annotated, 'BNJM' and not 'MTJM'. This is categorised as 1x _Referents_ error. In addition, **B** does not link his *C12* to his *C13* and is therefore categorised as 3x _Annotator Error_.

    ------------------------------------------------------------

6. C15 unpaired **A**:

    * Of the four occurences of 'PNJ' (substantive, Mpl, 'face') - 17:2, 17:15, PNJ+K, suffix 2Msg; 17:9, 'PNJ RC<JM'; 17:13, PNJ+W, suffix 3Msg - **A** has annotated the two PNJ mentions that have the 2Msg suffix. They both refer to the face of God. **B** adds the two mentions to his singletons. These two misses are categorised as 4x an _Annotator Error_ (2x for the class diff, 2x for the singletons).

    ------------------------------------------------------------

7. S - S $d_{j} \approx 0.3818$:

    * Most of the differences between **A** and **B**'s annotations have been discussed, a few mentions are still left open however.
    * In **A**'s diff 'LJLH', '>ZN' and '>JCWN BT&<JN' are of interest. 'LJLH' is a time phrase, and should be annotated. '>ZN' ('ear') matches **B**'s erroneously annotated 'ZN': he forgot the aleph ('>'). The same goes for '>JCWN BT&<JN' ('pupil of the eye'), which matches **B**'s 'JCWN BT&<JN': here the aleph is also missing. That makes 6x an _Annotator Error_.
    * In **B**'s diff 'TMK', 'MMTQWMMJM', 'YDQ' and 'HQJY' are of interest. 'TMK' and and 'HQJY' are infinitives, 'MMTQWMMJM' is the same as **A**'s 'MTQWMMJM' in his *C7*. **B** has incorrectly annotated an extra 'M', which is a preposition. That leaves the NP 'YDQ' which **A** has incorrectly not annotated as mention. That makes 8x an _Annotator Error_.


#### Conclusion Psalm 17

The low IAA measure of 0.4278 is a result of: 

* 70x _Annotator Error_, 
* 5x of _Referents_ and 
* 6x cases of _Genuine Ambiguity_. 

The _Annotator Error_ s and _Referents_ errors can be corrected, the interpretation cannot. 

### 4.4 Psalms_020.iaa 

`Psalms_020.iaa` has an IAA measure of $d_{j} \approx 0.4022$. As can be seen below none of the matched classes are perfectly similar, therefore the differences between all classes will be studied. 

In [22]:
ps020_df = MakeTable('iaa-files/Psalms_020.iaa')

ps020_df

,ann_A,ann_B,L,M,R,D,d
0,C1,C1,0,10,1,1,0.0909
1,C2,C3,1,10,2,3,0.2308
2,C3,C5,2,12,13,15,0.5556
3,C4,C2,1,1,1,2,0.6667
4,C7,C4,0,3,2,2,0.4000
5,S,S,1,19,0,1,0.0500
6,C5,-,2,0,0,2,1.0000
7,C6,-,11,0,0,11,1.0000


In [23]:
retrieve_ann('chris_A/Psalms_020.ann', 'gyus_B/Psalms_020.ann')

ann_A	ann_B	L	M	R	D	d	

------------------------------------------------------------
C1	C1	0	10	1	1	0.0909
------------------------------------------------------------
A	['K', 'K', 'K', 'K', 'K', 'K', 'K', 'K', 'K', 'K']	

B	['K', 'K', 'K', 'K', 'K', 'K', 'K', 'K', 'K', 'K', 'K']	

A diff	[]	

B diff	[('K', 10)]	

------------------------------------------------------------
C2	C3	1	10	2	3	0.2308
------------------------------------------------------------
A	['NRNNH', 'NW', 'NDGL', '>NXNW', 'NW', 'NZKJR', '>NXNW', 'QMNW', 'NT<WDD', 'NW', 'QR>NW']	

B	['NRNNH', 'NW', 'NDGL', '>NXNW', 'NW', 'NZKJR', '>NXNW', 'QMNW', 'NT<WDD', 'NW', 'QR>', 'NW']	

A diff	[('QR>NW', 10)]	

B diff	[('QR>', 11), ('NW', 12)]	

------------------------------------------------------------
C3	C5	2	12	13	15	0.5556
------------------------------------------------------------
A	['J<N', 'JHWH', 'JFGB', 'CM05 >LHJ J<QB', 'JCLX', 'JS<D', 'JZKR', 'JDCNH', 'JTN', 'JML>', 'K', 'CM&>LHJ', 'JML>', 'JHWH']	

B	['J<N', 'JHWH'

1. C1 - C1 $d_{j} \approx 0.0909$:

    * **A** and **B** both distinguish a 'K' (suffix, 2Msg, 'you') class. **A** annotates one 'K' less than **B**, and adds it to his *C3*. **A**'s *C3* is a 'JHWH', God, class. Though the text is a bit ambiguous - it could refer to 'JHWH' - the 'K' should have been added to *C1*. This is categorised as 2x an _Referents_ error (1x for the diff for this match, and 1x for **A**'s *C3*).
    
    -----------------------------------------------------------
    
2. C2 - C3 $d_{j} \approx 0.2308$:

    * In 20:10 **A** has annotated 'QR>NW' where **B** has 'QR>' and 'NW'. 'QR>NW' is an infinitive construct, 'to call' and functions as a predicate with subject suffix. The annotation guidelines state that infinitives with a subject suffix are marked as one mention. The disagreement is therefore noted as 3x an _Annotator Error_.
    
    -----------------------------------------------------------
    
3. C3 - C5 $d_{j} \approx 0.5556$, || C4 - C2 $d_{j} \approx 0.6667$ || and *C6* unpaired **A**: 

    * **B** has annotated a larger class than **A**. The diff of the latter indicates two differences: 'JTN' and 'K'. 'K' has already been discussed. 'JTN' in 20:5, is a verb, 3Msg of 'to give', **B** has forgotten to annotate the *Yod* ('J'), which results in the annotation of 'TN'. This is 2x _Annotator Error_. 
    * Except for 'TN', as noted, and 'MLK', **B**'s diff is much larger and corresponds to **A**'s unpaired *C6*. 'MLK' is dealt with below.
    * **B** annotates for *C5* in 20:7 'HWCJ<', **A** 'HWCJ'. 'HWCJ<' (verb, 2Msg, 'to help') is the right form, **A** has forgotten to annotate the *Ayin* ('<'). This is 2x _Annotator Error_ (1x for the diff in C3 - C5, 1x for the diff in **A**'s *C6*).
    * **B** annotates 'J<NH' in 20:7 ('J<NH', 17), the correct annotation of the mention should be 'J<N' (verb, 3Msg, 'to answer') however. The 'H' belongs to the suffix 'HW', 3Msg. This also explains **B**'s difference with **A** in their *C4* - *C2* match. **B**'s 'W' should be 'HW' there. These are 3x an _Annotator Error_. 
    * Both **A** and **B** have found that their *C3* and *C5* respectively can be identified as 'YHWH'. Since **A**'s  unpaired *C6* also contains the name 'JHWH', **A** has probably made an linking error. This can be counted as 2x 11x = 24x _Annotator Error_ (11x from **B**'s *C5* diff and 11x for **A**'s *C6* diff).
    
    -----------------------------------------------------------
    
4. C7 - C4 $d_{j} = 0.4$ || and *C5* unpaired **A**:

    * **B** has added '>LH' 2x to his *C4*, where **A** has added 2x '>LH' to a separate class *C5*. '>LH' (Cpl, 'these') in 20:8 both function as subject in a demonstrative pronoun phrase, hence **B**'s correct annotation of these mentions in one class. The demonstratives refer to those who trust in chariots and horses, but not in the name of God. 4x an _Annotator Error_.
    
    -----------------------------------------------------------
    
5. S - S $d_{j} = 0.05$:

    * **A** has annotated 'MLK' (substantive, 'king') as singleton, and **B** has not. **B** has added 'MLK' to his *C5*, which is the 'YHWH' class. **A** finds 'MLK' does not refer to God. It can be classified as 2x an _Referents_ (1x for *S*, 1x for *C3* - *C5*) error.


#### Conclusion Psalm 20

The lower IAA measure is a result of:

* 4x a _Referents_ error and 
* 38x an _Annotator Error_ 

and there are no cases of *Genuine Ambiguity*. That means that if the errors are corrected **A** and **B** agree on who is who on this text.

### 4.5 Psalms_032.iaa 

`Psalms_032` has an IAA measure of $d_{j} \approx 0.3983$. What immediately strikes the eye when taking a look at the dataframe below is that **B** has 5 additional unpaired classes. 

In [24]:
ps032_df = MakeTable('iaa-files/Psalms_032.iaa')

ps032_df

,ann_A,ann_B,L,M,R,D,d
0,C1,C2,0,3,0,0,0.0000
1,C2,C4,0,2,0,0,0.0000
2,C3,C8,14,13,0,14,0.5185
3,C4,C5,0,2,0,0,0.0000
4,C5,C6,0,2,0,0,0.0000
5,C6,C10,0,3,0,0,0.0000
6,C7,C11,0,2,0,0,0.0000
7,C8,C13,0,3,2,2,0.4000
8,C9,C1,1,16,1,2,0.1111
9,C10,C14,1,1,1,2,0.6667


We select therefore all matched classes that have a $d_{j}$ of .10 or higher:

In [25]:
ps032_df.loc[(ps032_df['d'] >= 1/10)]

,ann_A,ann_B,L,M,R,D,d
2,C3,C8,14,13,0,14,0.5185
7,C8,C13,0,3,2,2,0.4000
8,C9,C1,1,16,1,2,0.1111
9,C10,C14,1,1,1,2,0.6667
10,C11,C15,1,1,1,2,0.6667
12,S,S,4,21,6,10,0.3226
13,-,C3,0,0,5,5,1.0000
14,-,C7,0,0,2,2,1.0000
15,-,C12,0,0,3,3,1.0000
16,-,C16,0,0,3,3,1.0000


In [26]:
retrieve_ann('chris_A/Psalms_032.ann', 'gyus_B/Psalms_032.ann')

ann_A	ann_B	L	M	R	D	d	

------------------------------------------------------------
C1	C2	0	3	0	0	0.0
------------------------------------------------------------
A	['>CRJ >DM', 'W', 'W']	

B	['>CRJ >DM', 'W', 'W']	

------------------------------------------------------------
C2	C4	0	2	0	0	0.0
------------------------------------------------------------
A	['BLW', '<YM']	

B	['BLW', '<YM']	

------------------------------------------------------------
C3	C8	14	13	0	14	0.5185
------------------------------------------------------------
A	['HXRCTJ', 'J', 'J', 'J', 'J', 'J', '>WDJ<', 'J', 'KSJTJ', '>MRTJ', '>WDH', 'J', 'J', 'J', 'NJ', 'NJ', 'K', 'K', 'TLK', 'K', 'THJW', 'K', 'FMXW', 'GJLW', 'YDJQJM', 'HRNJNW', 'KL&JCRJ&LB']	

B	['J', 'KSJTJ', '>MRTJ', '>WDH', 'J', 'J', 'J', 'NJ', 'NJ', 'K', 'K', 'TLK', 'K']	

A diff	[('HXRCTJ', 0), ('J', 1), ('J', 2), ('J', 3), ('J', 4), ('J', 5), ('>WDJ<', 6), ('THJW', 20), ('K', 21), ('FMXW', 22), ('GJLW', 23), ('YDJQJM', 24), ('HRNJNW', 25), ('KL&JCRJ

1. C3 - C8 $d_{j} \approx 0.5185$ || *C3*, *C7*, *C12*, *C16* and *C17* unpaired B:

    * The IAA meausure can be explained for a great part by a discussion of the *C3* - *C8* match in combination with **B**'s unpaired classes. 
    * Both **A** and **B** recognise an 'I' (1Csg)/ 'You' (2Msg) class, but **A** connects more mentions to his class than **B** to his. The 'I' person starts to speak in 32:3 and stops in 32:8 where YHWH speaks in the 1Csg. Since the 'I' person in 32:3 -- 32:8 is probably the same person, e.g. a supplicant, **B** seems to have made a linking error, hence creating his unpaired *C3*. **B**'s *C3* contains 5 1Csg references, but only the first 3 ('HXRCTJ', 'J', 'TJ') can be counted in 32:3. The other two ('J', 'J') occur in 32:4. **B**'s *C7* contains only 2 ('J', suffix; '>WDJ<', verb) of the 8 1Csg mentions from 32:5, the other 6 are added to his matched *C8* class. This division of these 8 1Csg mentions and **B**'s unpaired *C3* and *C7* are probably caused by making linking errors in brat during annotation. In addition, 'TJ' is an erroneous annotation of the suffix 'J' of the substantive 'C>GT' ('roaring'). This results in a difference in **A**'s and **B**'s *S*: ('C>GT', 8) versus ('C>G', 9). We count 17x _Annotator Error_ (9x for the missed 1Csg mentions, 8x for the unpaired diffs).
    * **A** also adds ('THJW', 20) and ('K', 21) to his *C3* class. Both mentions occur in 32:9. In the previous verse, 32:8, a 2Msg is addressed. The sudden change to the 2Mpl of 'THJW' (verb, 'to be') in 32:9 is hard to follow and ambiguous: is the 2Msg the same as the 2Mpl? Probably not, but who or what they are remains unclear. We count 4x _Genuine Ambiguity_ (2x for **A**'s *C3* diff, 2x for **B**'s *C12* diff). The infinitive construct ('BLWM', 'to curb') annotated by **B** in his unpaired *C12* has no lexical identity with another occurrence of the same verb, it is therefore 1x _Annotator Error_. 
    * **A** adds ('FMXW', 22), ('GJLW', 23), ('YDJQJM', 24), ('HRNJNW', 25) and ('KL&JCRJ&LB', 26) to his *C3*, all mentions occur in 32:11. **B** divides these 5 mentions in two classes: *C16* is a 'YDJQJM' (vocative NP, Mpl, 'rigteous/just') class, *C17* is a 'KL&JCRJ&LB' (vocative NP, 'all the upright of heart') class. The 'righteous' and 'all the upright of heart' seem to be two different entities, therefore **B** is right in annotating two classes. We count 2x a linking error for **A**: 1 for the linking of 'YDJQJM' and 'KL&JCRJ&LB' and 1 for linking those mentions to **A**'s *C3*. These 5 mentions are counted as 5x _Annotator Error_.

    -----------------------------------------------------------
    
2. C8 - C13 $d_{j} = 0.4$:

    * These mentions occur in 32:9. **B** links two more mentions, ('BJN', 2) and ('QRB', 4), to his *C13* than **A** to his *C8*. 'BJN' and 'QRB' are infinitives in construct state with no lexical identity elsewhere. 2x _Annotator Error_.
    
    -----------------------------------------------------------
    
3. C9 - C1 $d_{j} \approx 0.1111$: 

    * The difference between **A** and **B** is a Yod ('J') of 'JHWH', which **B** forgot to annotate. 2x _Annotator Error_. 
    
    -----------------------------------------------------------

4. C10 - C14 $d_{j} \approx 0.6667$:

    *  The difference between **A** and **B** is a paragogic Nun ('N') of 'JSWBBN', which **B** forgot to annotate. 2x _Annotator Error_.

    -----------------------------------------------------------

5. S - S $d_{j} \approx 0.3226$: 

    * The differences between **A**'s and **B**'s annotations of singletons can be explained as follows. 
    * **B**'s 'BRWX' should be 'RWX' (substantive, sg, 'spirit/wind'). **B** erroneously annotated the preposition 'B': 2x  _Annotator Error_.
    * **B**'s 'C>G' has been disccused. 
    * **A**'s ('JWMM WLJLH', 11) in 32:4 is a time phrase 'by night and day', which should defined as a named entity in the guidelines: 2x _Guidelines_. 
    * **A** annotated ('MK>WBJM', 25) where **B** has ('BJM MK>WBJM', 27). The whole phrase in 32:10 is 'RBJM MK>WBJM' (Subject NP, Mpl, 'many pains'). 'RBJM' is an adjective, and should not be annotated as mention. 2x  _Annotator Error_. 
    * 'MY>' ('to find') in 32:6 and 'PLV' ('to escape') are both an infinitive construct and have no lexical identity elsewhere. 2x _Annotator Error_. 
    
#### Conclusion Psalm 32

The IAA measure is the result of:

* 35x _Annotator Error_ and 
* 4x _Genuine Ambiguity_. 

The 35 annotator errors can be fixed. In the analysis of Psalm 32 but also in previous analyses, the difference in some of the annotations between **A** and **B** was a result of linking errors and not necessarily the erroneous annotation of mentions. The consequence of a linking error however is that coreference classes are built up and matched differently, but also that the interpretation of the text is (essentially) different. In some sense, the IAA measures express this difference in interpretation. In the analyses to come the linking error will be distinguished, but is not added to the categorisation of disagreement types. 

### 4.6 Psalms_070.iaa 

The IAA measure for `Psalms_070` is $d_{j} \approx 0.3818 $. Leaving the classes with 0 difference out of consideration, we zoom in on all classes with $d_{j} = 0.06 $ or higher. 

In [27]:
ps070_df = MakeTable('iaa-files/Psalms_070.iaa')

ps070_df

,ann_A,ann_B,L,M,R,D,d
0,C1,C1,2,6,3,5,0.4545
1,C2,C2,0,3,0,0,0.0000
2,C3,C3,1,2,1,2,0.5000
3,C4,C4,0,3,0,0,0.0000
4,C5,C5,2,2,3,5,0.7143
5,C6,C6,1,13,0,1,0.0714
6,S,S,3,5,3,6,0.5455
7,C7,-,2,0,0,2,1.0000


In [28]:
ps070_df.loc[(ps070_df['d'] >= .06)]

,ann_A,ann_B,L,M,R,D,d
0,C1,C1,2,6,3,5,0.4545
2,C3,C3,1,2,1,2,0.5000
4,C5,C5,2,2,3,5,0.7143
5,C6,C6,1,13,0,1,0.0714
6,S,S,3,5,3,6,0.5455
7,C7,-,2,0,0,2,1.0000


In [29]:
retrieve_ann('chris_A/Psalms_070.ann', 'gyus_B/Psalms_070.ann')

ann_A	ann_B	L	M	R	D	d	

------------------------------------------------------------
C1	C1	2	6	3	5	0.4545
------------------------------------------------------------
A	['NJ', 'J', 'J', 'J', '>NJ', 'J', 'J', 'J']	

B	['DWD', 'NJ', 'TJ', 'J', 'TJ', '>NJ', 'J', 'J', 'J']	

A diff	[('J', 2), ('J', 3)]	

B diff	[('DWD', 0), ('TJ', 2), ('TJ', 4)]	

------------------------------------------------------------
C2	C2	0	3	0	0	0.0
------------------------------------------------------------
A	['JBCW', 'JXPRW', 'MBQCJ']	

B	['JBCW', 'JXPRW', 'MBQCJ']	

------------------------------------------------------------
C3	C3	1	2	1	2	0.5
------------------------------------------------------------
A	['JSGW', 'JKLMW', 'XPYJ R<T']	

B	['JSGW', 'JKLMW', 'XPYJ R<']	

A diff	[('XPYJ R<T', 2)]	

B diff	[('XPYJ R<', 3)]	

------------------------------------------------------------
C4	C4	0	3	0	0	0.0
------------------------------------------------------------
A	['JCWBW', 'M', '>MRJM']	

B	['JCWBW', 'M', '>MRJM'

1. C1 - C1 $d_{j} \approx 0.4545$:

    * For this matching there are three differences in annotations between **A** and **B**. **B** erroneously annotated an extra Taw ('T') with the 1Csg suffixes ('TJ', 2) and ('TJ', 4). The suffixes should be 'J'. The named entity 'DWD' ('David') occurs in the title of the Psalm, and should not be linked to the rest of the Psalm. **B**'s linking of David also explains the difference in *S*, where **A** annotated David as singleton. We count 6x an _Annotator Error_.
    
    -----------------------------------------------------------

2. C3 - C3 $d_{j} = 0.5$:

    * The difference between **A** and **B** is one Taw: 'T'. The 'T' is erroneously ommitted: 2x _Annotator Error_. 

    -----------------------------------------------------------
    
4. C5 - C5 $d_{j} \approx 0.7143$ || and *C7* unpaired **A**:

    * **B** annotates 'KL&MBQCJ' in 70:5 as one mention: 'all who seek', versus **A** who annotates two mentions. 'KL' is a subject NP of the predicate participle 'MBQCJ' (Mpl). They should be annotated as two separate mentions. 3x _Annotator Error_. 
    * **B** also annotates ('>HBJ JCW<T', 5) in 70:5 as one mention, versus **A** who links 'J>MRW' to '>HBJ' in his unpaired *C7*. '>HBJ JCW<T', translated as 'those who love salvation', are two mentions. '>HBJ' is a verbal phrase, participle Mpl; 'JCW<T' is a determined object NP (determined by a 2Msg suffix). 2x _Annotator Error_. In addition, **B** links '>HBJ JCW<T' and 'J>MRW' (verb, 3Mpl, 'to say') to his *C5* hence making 'all who seek' and 'those who love salvation' the same entity. Though the parallelism could suggest the mentions refer to the same entity, it is not clear that they are actually the same. Therefore, similar to the 'YDJQJM' and 'KL&JCRJ&LB' case in Ps 32, they should be annotated as two different entities like with **A**'s unpaired *C7*: 'J>MRW' and '>HBJ'. As a result of this annotation 'JCW<T' is added to **A**'s singleton class. 5x an _Annotator Error_ (2x for **A**'s *C7*, 2x **B**'s *C5*, 1x for *S*). 

    -----------------------------------------------------------

5. C6 - C6 $d_{j} \approx 0.0714$: 

    * The difference between **A** and **B** is 'HYJL' (infinitive, 'to deliver') found in 70:2. 'HYJL' functions as predicate with an object suffix 1Csg which is naturally annotated seperately from the infinitive. While the annotation guidelines dictate that infinitives with an object suffix are marked separately as two mentions, the guidelines do not say how they should be coreferenced when there is no lexical identity of the verb. This is categorised as 2x _Guidelines_. **B**'s correct annotation causes 'HYJL' to be added to his singleton class. 
    
    -----------------------------------------------------------

6. S - S $d_{j} \approx 0.5455$: 

    * All differences in this matching have been discussed in relation to other class matches, except for **A**'s ('<ZRT', 2) and **B**'s ('<ZR', 4). **B** forgot to annotate the *Taw*, which was annotated with the suffix in the C1 - C1 matching. 2x an _Annotator Error_. 
    
#### Conclusion Psalm 70

* 20x _Annotator Error_ and 
* 2 _Guidelines_. 

## 5. Numbers 

As stated in the introduction in §1.1 the hypothesis for the comparison between coreference annotations in poetry and narrative texts is that annotators **A** and **B**, or any annotator for that matter, will agree more on their annotations of the narrative texts than on the poetic texts. This has to do with a number of assumptions:

* Poetic structure is harder to understand than narrative because of shorter word forms, hapax legomena etc. 
* It is often more clear who is who in BH narrative texts than in poetry, since persons in narrative are identified with all kinds of epithets and designations. In BH poetry persons often remain unidentified. 
* The structure of poetry versus narrative text and the absence of identified persons in poetry versus identified persons in narrative would therefore make the annotation of coreference an easier task in narrative texts. 

Numbers contains narrative text, let's see if a comparison of Numbers 8--10 can shed any light on these assumptions. 

First we pull in the IAA measures of Numbers, and sort the $d_{j}$ like we did with the Psalms in ascending order:

In [30]:
! sort -k 7n,7 iaa-files/total_numbers

Numbers_009.iaa	-	79	156	80	159	0.5048
Numbers_008.iaa	-	82	162	86	168	0.5091
Numbers_010.iaa	-	98	174	100	198	0.5323


In [31]:
! python3 acc.py iaa-files/total_numbers iaa-files/total_psalms

iaa-files/total_numbers	-	259	492	266	525	0.5162
iaa-files/total_psalms	-	182	622	194	376	0.3768


Interesting, the IAA measures for the annotations of **A** and **B** indicate that they disagree more than they did on the Psalms: 0.5249 for Numbers versus 0.3768 for the Psalms (0.1418 difference). The Numbers measures do not differ greatly from each other (maximally 0.04). Let's start with the highest IAA measure, i.e. `Numbers_008`.

### 5.1 Numbers_008.iaa

We make a table to see on how many classes **A** and **B** agree and disagree. Then we take a closer look at all classes on which **A** and **B** disagree. 

In [32]:
nu008_df = MakeTable('iaa-files/Numbers_008.iaa')

nu008_df

,ann_A,ann_B,L,M,R,D,d
0,C1,C23,1,2,5,6,0.7500
1,C2,C11,0,25,1,1,0.0385
2,C3,C2,0,2,0,0,0.0000
3,C4,C3,0,3,0,0,0.0000
4,C5,C4,0,2,0,0,0.0000
5,C6,C5,1,3,5,6,0.6667
6,C7,C10,5,0,5,10,1.0000
7,C8,C17,2,0,3,5,1.0000
8,C11,C6,0,3,0,0,0.0000
9,C12,C7,1,1,2,3,0.7500


In [33]:
nu008_df.loc[(nu008_df['d'] >= .0380)]

,ann_A,ann_B,L,M,R,D,d
0,C1,C23,1,2,5,6,0.7500
1,C2,C11,0,25,1,1,0.0385
5,C6,C5,1,3,5,6,0.6667
6,C7,C10,5,0,5,10,1.0000
7,C8,C17,2,0,3,5,1.0000
9,C12,C7,1,1,2,3,0.7500
12,C18,C22,5,37,26,31,0.4559
13,C19,C12,0,3,1,1,0.2500
15,C23,C9,0,10,1,1,0.0909
18,C27,C15,1,1,3,4,0.8000


In [34]:
retrieve_ann('chris_A/Numbers_008.ann', 'gyus_B/Numbers_008.ann')

ann_A	ann_B	L	M	R	D	d	

------------------------------------------------------------
C1	C23	1	2	5	6	0.75
------------------------------------------------------------
A	['YB>', 'YB>', 'YB> H<BDH']	

B	['<BD', 'YB>', '<BDT >HL MW<D', 'YB> H<BDH', 'MCMRT', '<BDH', 'BMCMRT']	

A diff	[('YB>', 1)]	

B diff	[('<BD', 0), ('<BDT >HL MW<D', 2), ('MCMRT', 4), ('<BDH', 5), ('BMCMRT', 6)]	

------------------------------------------------------------
C2	C11	0	25	1	1	0.0385
------------------------------------------------------------
A	['MCH', 'DBR', '>MRT', 'MCH', 'MCH', 'MCH', 'QX', 'VHRT', 'T<FH', 'VHR', 'HZH', 'TQX', 'HQRBT', 'HQHLT', 'HQRBT', '<FH', 'H<MDT', 'HNPT', 'HBDLT', 'VHRT', 'HNPT', 'MCH', 'MCH', 'MCH', 'T<FH']	

B	['MCH', 'DBR', '>MRT', 'MCH', 'MCH', '<FH', 'MCH', 'QX', 'VHRT', 'T<FH', 'VHR', 'HZH', 'TQX', 'HQRBT', 'HQHLT', 'HQRBT', '<FH', 'H<MDT', 'HNPT', 'HBDLT', 'VHRT', 'HNPT', 'MCH', 'MCH', 'MCH', 'T<FH']	

A diff	[]	

B diff	[('<FH', 5)]	

----------------------------------------

1. C1 - C23 $d_{j} = 0.75$ || and *C37* unpaired A: 
    
    *  The matched classes can be identified as 'the work in the tent of meeting'. The differences between **A** and **B** seem to be caused mainly by a difference in the understanding of the descriptions of the different tasks of the retired Levites in 8:24-26.
    * **B** adds '<BD' (infinitive, 'to work') in 8:22 to his *C23*; **A** annotates a separate class *C22* in which '<BD' is added. **A** finds that '<BD' refers to another entity. 2x _Referents_. 
    * **B** adds `<BDT >HL MW<D` (object NP, 'work in the tent of meeting') in 8:24 to his *C23*, **A** adds this NP to his singletons. It is possible to corefer this object NP with the other descriptions of the work in the tent of meeting. **A** considers the NP to have no referent, it is therefore counted as 2x _Referents_.  
    * Where **A** adds another 'YB>' to his *C1*, **B** does not. 'YB> (inf, 'to serve') and 'YB>' (substantive, 'service') in 8:24 should be coreferenced since the infinitive shares a lexical identity with the following substantive. **B**'s 'YB>' ends up in his *C20*. 2x _Annotator Error_. 
    * **B** adds 'MCMRT' and 'BMCMRT' to his *C20*, where **A** adds them, without the extra 'B', to a seperate class *C37*. 'MCMRT' (substantive, Fsg) means 'guard-post'. In 8:26 the extra 'B' is a preposition, and should not be annotated as mention: 2x _Annotator Error_. 'MCMRT' can be considered as part of the task that the Levites carry out, but can also describe the actual 'work in the tent of meeting'. Because of this ambiguity we count 2x _Genuine Ambiguity_.
    * **B** rightly adds '<BDH' to his *C20*, it refers to the work in the tent of meeting. **A** adds '<BDH' to his singletons. 2x _Annotator Error_. 

    ------------------------------------------------------------

2. C2 - C11 $d_{j} \approx 0.0385$: 

    * The only difference in this matching is '<FH' (verb, 3Msg, 'to make') which occurs in 8:4 and 8:12. '<FH' has been annotated by **B** at the beginning of the chain so it probably is the occurence in 8:4. The question is to whom the 3Msg of the verb refers. **B** thinks it refers to the previous object Moses ('MCH'): 'thus he made the lamp stand', hence his annotation. **A**' adds '<FH' to his singletons, probably considering the 3Msg referring to the lampstand: 'thus it was made the lampstand'. 2x _Referents_. 
    
    ------------------------------------------------------------
    
3. C6 - C5 $d_{j} \approx 0.6667$ || C7 - C10 $d_{j} = 1.0$: 

    * The difference in annotations is caused by differing interpretations on what refers to the 'M<FH HMNRH' (NP, 'lamp stand') in 8:4. 
    * **A** erroneously omits 'ZH' (demonstrative subject NP, 'this'). 'ZH' as subject refers to the lamp stand. **A** adds  'ZH' to his *C7*, where he considers it to refer to 'MQCH ZHB' ('hammered gold work'). 2x _Annotator Error_ (1x for **A**'s *C7*, 1x for **A**'s miss in *C6*). 
    * **A**'s *C7* contains mentions that refer to 'hammered gold work'. Especially 'JRK' ('shaft', 'upper part') and 'PRX' ('bud') are of interest. They are both adjunct phrases, and should not have been added to a class by **A** and **B**. 'MQCH ZHB' and 'MQCH' describe the quality of the lamp stand, but are not mentions that refer to the entity 'lamp stand'. 4x _Annotator Error_. **A** has also forgotten to add the second mention 'MNRH' to his class, thus adding it to his *S*. Both **A** and **B** have made 2x an _Annotator Error_. The class should have been annotated as: ['ZH', '<FH HMNRH', 'H', 'H', 'HW>']. 
    * **A** forgot to annotate the 'Mem' in '<FH HMNRH', this is 1x _Annotator Error_. 
    
    ------------------------------------------------------------
    
3. C7 - C10 $d_{j} = 1.0$ || C8 - C17 $d_{j} = 1.0$ || C31 - C1 $d_{j} = 0.6$ || C9 unpaired A: 

    * **A** forgot to link 'HR>H' and 'JHWH' (*C8*) to his *C34*. This explains at least two ('HR>H', 0) and ('JHWH', 1) of the differences between **A**'s and **B**'s C34 - C24 match. 4x _Annotator Error_.
    * **A** forgot to link 'JDBR' and 'JHWH' (*C9*) to his *C34*. This explains at least two ('JDBR', 7) and ('JHWH', 8) of the differences between **A**'s and **B**'s C34 - C24 match. 4x _Annotator Error_.
    * **B** annotates three classes (*C10*, *C17* and *C1*) that can be identified as '>HRN' ('Aaron'), **A** annotates one 'Aaron' class (*C31*). The occurrences of Aaron in Numbers 8 give no reason to separate them into different classes. **B** has therefore probably made three linking errors (one for each class). This means that they are counted as 5+3+6+15= 29x _Annotator Error_. 

    ------------------------------------------------------------

4. C12 - C7 $d_{j} = 0.75$:

    * **A**'s erroneously annotated 'LWLH' of the verb 'BLL' (participle, Fsg, 'to moisten'), he forgot a 'Bet' ('B'). 2x _Annotator Error_.
    * Unlike **B**, **A** added 'MNXT' to his singletons. The translation of **B**'s *C7* is: 'the offering of wheat , that is moisten'. **B** is right in his intepretation, 2x _Annotator Error_. 

    ------------------------------------------------------------

5. C18 - C22 $d_{j} \approx 0.4559$ || C32 - C18 $d_{j} = 0.5$ || C33 - C19 $d_{j} \approx 0.7143$ || C36 - C21 $d_{j} = 0.5$ || and *C17*, *C22* and *C29* unpaired A: 

    * The classes in the C18 - C22 match can be identified as 'LWJM' (substantive, 'Levites'). 
    * **A**'s diff indicates ('HVHRW', 9), where **B** has ('WHVHRW', 9). 'HVHRW' (verb, 3Cpl, 'be clean') in 8:7 should be annotated without the 'Waw', as **A** has done. 2x _Annotator Error_. 
    * **A** annotates ('M', 19), where **B** annotates ('HM', 20). **B** erroneously annotated the additional 'H'. 2x _Annotator Error_.
    * **A**' adds ('NTNJM', 29) and ('NTNJM', 30) to his *C18*. Both mentions, occurring in 8:16, are participles Mpl 'to give', and refer to the Levites 'who are given'/'gifts'. The substantive ('NTNJM', 37) in 8:1 (Mpl, 'the gifts') which also refers to Levites as gifts, is added by **B** to his *C22*. **A** adds the substantive 'NTNJM' to his singletons. **B** links the participles 'NTNJM' to his *C24* which is a 'JHWH' ('God') class. The subject and referent of the Mpl of the two participles in 8:16 is the Levites, and not God. 4x _Annotator Error_. **A** should have linked the substantive 'NTNJM' to his *C18* however: 2x _Annotator Error_. 
    * **B** annotates a considerable amount of mentions more than **A**. Ten mentions that refer to the Levites can be found in **A**'s *C29* unpaired class which is probably the result of a linking error: 20x _Annotator Error_. Within these 20 errors **A** erroneously annotated 'TXV>W' instead of **B**'s ('JTXV>W', 42). **B** erroneously annotated ('TM', 53) instead of 'M', like **A**.
    * Depending on the interpretation of the referent of 2x 'J<BD' (verb, 3Msg, 'to work/serve') in 8:25-26, the infinitives '<BD' (4x) can be corefered with the Levites class or not. If 'J<BD' refers to the Levites, the infinitives are annotated as mention and corefered with the Levites. If 'J<BD' does not refer to the Levites, but to the retired Levites, the infinitives should not be annotated as mention and coreferred. 
    * While **A** annotates a separate class *C22* with the 4 infinitives and 1 substantive ('work), **B** adds two infinitives and the substantive to his *C22*. The substantive in 8:22 occurs with a suffix 3Mpl, which should be annotated as '<BDT' and 'M'. **B** erroneously annotates an extra 'Taw' resulting in **B**'s diff: ('<BD', 52) and ('TM', 53). This is 2x _Annotator Error_. The question of interpretation is to whom the 3Msg of 'J<BD' refers (**B**'s ('J<BD', 61)): to the Levites or the retired Levites in 8:25 ('BN XMCJM CNH', adjunct NP, a son/man of fifty years)? **A**'s annotation of *C36* points to the latter: 'From (a son/man of fifty years ['BN XMCJM CNH']) (he will return ['JCWB']) from regular work and (he will work ['J<BD']) no more. (He serves ['CRT']) (his ['W']) brothers ... and (he does not work ['J<BD'])'. **B** links however 'CRT', 'W' and 'J<BD' to the Levites (*C22*) class. **B** thus considers these three mentions to refer to the Levites, not to the retired Levites. **B** does link one 'J<BD' to his *C36*, but the verb has the same subject (retired Levites) as the 'J<BD' that **B** linked to *C22*. Though the retired Levites were part of the Levites, they are clearly referred here as a different group that is separate from the Levites serving in the tent of meeting. 6x _Annotator Error_. 
    * Now that the referent of 2x 'J<BD' is determined, **A**'s annotation of *C22* and **B**'s annotation of '<BD' in his *C22* can be discussed. The first three annotations of '<BD' in **A**'s *C22* are erroneous. The same goes for 3x '<BD' in **B**'s *C22*. They do not have any lexical identity with verbs or substantiva in their respective verses. According to that same principle '<BD' and '<BDT' can be coreferenced in a seperate class. 6x _Annotator Error_. 
    * **B**'s diff indicates that '>XJ' 'brother' was linked to his *C22*; **A** erroneously added '>XJ' to his singletons. '>XJ' refers to the Levites as group, **B** has rightly linked the mention. 2x _Annotator Error_. 
    * **B** annotated the infinitive ('CMR', 60) with no lexical identity. 1x _Annotator Error_. 
    * Of the four occurrences of 'TNWPH' (substantive, Fsg, 'offering') in Nu 8 **B** links one to *C18* since the Levites are presented to JHWH as a(n) (symbolic) offering. The other three mentions of 'TNWPH' are added to his singletons. **A** adds all four mentions to his unpaired *C17*. **A** should have added one of those mentions to his *C18*, but unlike **B** he is right in corefering the other three mentions of 'TNWPH'. That makes 2 + 6 = 8x _Annotator Error_.
    * **B** links '<FW' (verb, 3Cpl, 'to make') in 8:22 to his Levites class, i.e. *C22*. **A** adds the verb to his singletons. The 3Cpl refers to the preceding 'PNJ >HRN W PNJ BNJ', 'the face of Aaron and the face of (his) sons', and not the Levites. In the ETCBC representation the adjunct NP looks like this: 'LPNJ >HRN WLPNJ BNJW'. The annotator leaves out the prepositions in  brackets: (L)PNJ >HRN W (L)PNJ BNJ-W. The suffix -W is annotated separately, it refers to Aaron. The annotation guidelines exclude the annotation of prepositions within and as mentions. The annotation tool brat poses a problem here: nested mentions cannot be corefered, which can be seen in the *C32* - *C18* and *C33* - *C19* matchings. The suffix 'W' refers to 'PNJ >HRN and '<FW' refers to 'PNJ >HRN W PNJ BNJ'. Accordingly, **A** annotates 'PNJ BNJ' and 'PNJ BNJ' seperately in his *C33* and links 'PNJ >HRN' with corefering suffixes in his *C32*. '<FW' is added to **A**'s singletons. **B** links 2x 'PNJ BNJ' to his *C19* together with 'JKPR' (explicit subject is Aaron), 'W' and 'HM' (refers to the Levites). **B**'s ('KPR', 0), ('JKPR', 2), ('>HRN', 3), ('W', 5) and ('HM', 6) do not corefer with 'PNJ BNJ'. Nor does '<FW' corefer with the Levites. That means: 12x _Annotator Error_. Since brat's inflexibility with nested mentions, the 2x suffix 'W' can be referred to both 'PNJ >HRN' and '>HRN'. **B**'s ('W', 1), ('W', 3) in C32 are categorised as 2x _brat_. 
    
    ------------------------------------------------------------
    
6. C19 - C12 $d_{j} = 0.25$: 

    * **A** has not corefered one occurence of 'TWK BNJ JFR>L' ('the middle of the sons of Israel'). The mention is added to his singletons. 2x _Annotator Error_. 

    ------------------------------------------------------------
    
7. C23 - C9 $d_{j} \approx 0.0909$: 

    * **B** corefers 'GCT' (infinitive, 'to approach') with the 'BNJ JFR>L' 'sons of Israel' class. 'GCT' has no lexical identity with another occurence of the verb. 1x _Annotator Error_. 

    ------------------------------------------------------------
    
8. C27 - C15 $d_{j} = 0.8$: 

    * The 0.8 in this matching is the result of a differing mention annotation: `MCH W>HRN WKL&<DT BNJ&JFR>L` or split in three mentions. 
    * The annotation of `MCH W>HRN WKL& <DT BNJ&JFR>L`, nested subject NP's, 'Moses and Aaron and all of the children/sons of Israel', as one mention is possible. Though the annotation guidelines prefer that the NP is split up in three mentions since each nested NP is demarcated by a substantive or nomen proprium in absolute state, the preceding verb refers to the group in singular: it acts as one actor. The guidelines should be adjusted accordingly. 4x _Guidelines_. 

    ------------------------------------------------------------
    
9. C34 - C24 $d_{j} \approx 0.3939$: 

    * The classes in this match can be identified as 'JHWH', 'God'. Some of **B**'s differences in annotations compared to **A** have been discussed under point 3 above. 
    * **A**'s annotation of ('QXTJ', 8) is erroneous, he missed the 'L', cf. **B**'s annotation ('LQXTJ', 16). 2x _Annotator Error_.  
    * **A**'s ('HKTJ', 12) is an infinitive with subject suffix and is hence annotated as mention and corefered with his *C34*. **B** erroneously splits the mention in ('TJ', 20) (the 'TJ' should have been the suffix 'J') and rest of the word ('HK', 26) ends up in his singletons. 4x _Annotator Error_.
    * **B** annotates 3x '>MR'; **A** rightly annotates no '>MR', since it is an infinitive with no lexical identity. 3x _Annotator Error_. 

    ------------------------------------------------------------

10. *C15* unpaired A: 

    * **A**'s *C15* which contains two times 'XV>T' (in 8:8 and 8:12) refers to the 'sin offering' that the Israelites have to make. **B** adds them to his singleton class. Though both NP's are undetermined, they both refer to the same ritual offering. They therefore need to be corefered. 4x _Annotator Error_.

    ------------------------------------------------------------

10. *C16* unpaired A: 

    * **A** erroneously corefered 'R>C HPRJM' with suffix 3Mpl 'M'. 3Mpl refers to the Levites who are to lay their hands on 'the head of the young bull', 'R>C HPRJM'. **B** rightly adds 'R>C HPRJM' to his singletons. 4x _Annotator Error_. 

    ------------------------------------------------------------

11. *C21* unpaired A:

    * **A** erroneously corefered 'the land of Egypt' ('>RY MYRJM') with suffix 3Mpl ('M'). 3Mpl refers to the first borns that JHWH has taken. **B** adds '>RY MYRJM' to his singletons. 3x _Annotator Error_. 

    ------------------------------------------------------------
    
12. *C30* unpaired A:

    * **B** adds 2x 'BGDJ' (8:7 and 21) to his singletons. 'BGDJ', 'garment' occurs twice as determined object NP and should be corefered like **A**'s unpaired *C30*. 4x _Annotator Error_. 


13. S - S $d_{j} \approx 0.6087$: 

    * Most of the mentions have been discussed in relation to the matched classes, some mentions have not however. 
    * **A** annotates the second mention of 'hands' in 8:12 as 'JDJH' ('JDJH', 13). The 'H' is incorrect, as it is part of the suffix: 'HM'. 2x _Annotator Error_. 
    * **A** annotates ('HQDC', 24) where **B** annotates ('QDC', 32). **A** erroneously annotated the additional 'H', which is an article. 2x _Annotator Error_.


#### Conclusion Numbers 8
The count and categorisation of the differences in annotations between **A** and **B** are:

* 6x _Referents_
* 154x _Annotator Error_ 
* 2x _Genuine Ambiguity_
* 2x _brat_
* 4x _Guidelines_

Most differences are a result of _Annotator Error_ 's: linking errors, erroneous mention annotation, corefering infinitives etc. Some of these mistakes can be prevented by a more clear annotation process, and a different presentation of the data that should be annotated.

### 5.2 Numbers_010.iaa

The $d_{j}$ of **A**'s and **B**'s annotations $ \approx 0.5323 $. Let us analyse the differences per class. 

In [35]:
nu010_df = MakeTable('iaa-files/Numbers_010.iaa')

nu010_df

,ann_A,ann_B,L,M,R,D,d
0,C1,C11,5,3,0,5,0.6250
1,C2,C48,2,0,2,4,1.0000
2,C3,C2,3,0,3,6,1.0000
3,C4,C3,1,1,7,8,0.8889
4,C5,C6,1,2,1,2,0.5000
5,C6,C7,1,3,1,2,0.4000
6,C7,C5,0,4,4,4,0.5000
7,C8,C47,2,0,2,4,1.0000
8,C9,C39,2,0,2,4,1.0000
9,C10,C8,0,3,3,3,0.5000


In [36]:
nu010_df.loc[(nu010_df['d'] >= 1/14)]

,ann_A,ann_B,L,M,R,D,d
0,C1,C11,5,3,0,5,0.6250
1,C2,C48,2,0,2,4,1.0000
2,C3,C2,3,0,3,6,1.0000
3,C4,C3,1,1,7,8,0.8889
4,C5,C6,1,2,1,2,0.5000
5,C6,C7,1,3,1,2,0.4000
6,C7,C5,0,4,4,4,0.5000
7,C8,C47,2,0,2,4,1.0000
8,C9,C39,2,0,2,4,1.0000
9,C10,C8,0,3,3,3,0.5000


In [37]:
retrieve_ann('chris_A/Numbers_010.ann', 'gyus_B/Numbers_010.ann')

ann_A	ann_B	L	M	R	D	d	

------------------------------------------------------------
C1	C11	5	3	0	5	0.625
------------------------------------------------------------
A	['CTJ XYWYRT KSP', 'M', 'HJW', 'TQ<W', 'HN', 'XYYRWT', 'XYYRWT', 'XYYRT']	

B	['XYYRWT', 'XYYRWT', 'XYYRT']	

A diff	[('CTJ XYWYRT KSP', 0), ('M', 1), ('HJW', 2), ('TQ<W', 3), ('HN', 4)]	

B diff	[]	

------------------------------------------------------------
C2	C48	2	0	2	4	1.0
------------------------------------------------------------
A	['NW<DW', 'KL&H<DH']	

B	['NS< H>RN', 'NXH']	

A diff	[('NW<DW', 0), ('KL&H<DH', 1)]	

B diff	[('NS< H>RN', 0), ('NXH', 1)]	

------------------------------------------------------------
C3	C2	3	0	3	6	1.0
------------------------------------------------------------
A	['NW<DW', 'NFJ>JM', 'R>CJ >LPJ JFR>L']	

B	['JDBR', 'JHWH', '>MR']	

A diff	[('NW<DW', 0), ('NFJ>JM', 1), ('R>CJ >LPJ JFR>L', 2)]	

B diff	[('JDBR', 0), ('JHWH', 1), ('>MR', 2)]	

---------------------------------------

A diff	[('XNTNW', 3)]	

B diff	[('JS<W', 0), ('BNJ&JFR>L', 1), ('HM', 2), ('JS<W', 3), ('TM', 4), ('TM', 5), ('JS<W', 6), ('J>MR', 7), ('MCH', 8), ('NS<JM', 9), ('>NXNW', 10), ('KM', 11), ('JFR>L', 14), ('W', 15), ('J>MR', 16), ('NW', 18)]	

------------------------------------------------------------
C40	C40	1	2	0	1	0.3333
------------------------------------------------------------
A	['HJH', 'VWB', 'HW>']	

B	['HJH', 'HW>']	

A diff	[('VWB', 1)]	

B diff	[]	

------------------------------------------------------------
C41	C41	0	2	0	0	0.0
------------------------------------------------------------
A	['>RWN BRJT&JHWH', 'NS<']	

B	['>RWN BRJT&JHWH', 'NS<']	

------------------------------------------------------------
C42	C42	6	4	1	7	0.6364
------------------------------------------------------------
A	['JS<W', 'BNJ&JFR>L', 'HM', 'JS<W', 'JFR>L', 'JS<W', 'HM', 'HM', 'HM', 'NS<M']	

B	['JS<W', 'HM', 'HM', 'HM', 'M']	

A diff	[('JS<W', 0), ('BNJ&JFR>L', 1), ('HM', 2), ('JS<W', 3), ('JFR

1. C1 - C11 $d_{j} = 0.625$ || C2 - C48 $d_{j} = 1.0$ || C4 - C3 $d_{j} = 0.8889$ || C44 - C4 $d_{j} = 1.0$:

    * The differences between **A** and **B** are mainly the result of the question if 'CTJ XYWYRT KSP' in 10:2 (Fpl, 'two trumpets of silver') and 'XYYRWT' (Fpl, 'trumpets') 10:8-10 corefer, and what other mentions refer to these two NP's. 
    * **A** links 'HN' (suffix, 3Fpl) in 10:3 to his *C1*. **B** links 'HN' to 'MXNWT' (Fpl, 'camps') in 10:2 in his *C4*. The preposition 'B' of 'HN' suggests that it refers to the trumpets reflexively: 'when the trumpets themselves 'HN' are blown, **B** thinks otherwise. This is a case of 4x _Referents_. **A** adds 'MXNWT' to his  singletons. 2x _Referents_
    * **A** links 3x 'XYYRWT' to his *C1*; **B** links the three mentions into a separate class *C11*. Throughout the text all mentions of 'XYYRWT' refer to the same trumpets. 6x _Annotator Error_. 
    * **B** considers 'MQCH' (object NP, Fsg, 'hammered work') to refer to the trumpets that are hammered work and is thus a quality of the trumpets. The object NP does not refer to the trumpet as entity: 2x _Referents_. 
    * **B** links 'NW<DW' (verb, 3Cpl, 'to appoint') in 10:3 to his *C3*; **A** links the verb to his *C2* which has as explicit subject 'KL&H<DH' ('all of the gathering'). **B** adds 'KL&H<DH' to his *S*. **B** made 4x _Annotator Error_. 
    * **B** links the verb 'JTQ<W' (3Mpl, 'to blow') in 10:4 and 10:6 to his *C3*. **A** links the verb in 10:4 to his *C4* making the translation of the class: 'one is sounded'. **A** adds 'JTQ<W' in 10:6 to his singletons. Both **A** and **B** have misinterpreted 'JTQ<W' in 10:4, a translation would be: 'and if in one ('>XT', referring to one of the trumpets) they  blow ('JTQ<W')'. '>XT' (complement NP) is not subject of 'they blow', as **A**'s annotation suggests, nor does 'they blow' refer to the two trumpets as **B** suggests. 'they blow' probably refers to the anonymous, unreferred group of people that blow the trumpets. 5x _Annotator Error_. (2x for **A**'s *C4*, 2x for **B**'s 'JTQ<W', 1x for **A**'s *S*). 

    ------------------------------------------------------------
    
2. C2 - C48 $d_{j} = 1.0$ || C44 - C4 $d_{j} = 1.0$: 

    * **A**'s *C2* has aleady been discussed above. 
    * **B** links 'NS< H>RN' ('NS<': infinitive construct; H>RN: substantive, 'ark') for 10:35 in his *C48* to 'NXH' (infinitive, predicate subject 3Msg) in 10:36. **B** has made 3x _Annotator Error_ with his annotation of 'NS< H>RN'. 'NS<' is an infinitive with no lexical identity elsewhere in the text. '>RN', without the article 'H', as explicit subject should have been corefered with 'JHJ' (verb, 3Msg, 'to be') like in **A**'s *C44*. 'JHJ' now ends up in **B**'s singletons. Another 3x _Annotator Error_'s are counted for **B**'s erroneous diff for **A**'s diff in *C44*. **A** on his turn erroneously annotated the preposition 'B' with 'NXH'. 1x _Annotator Error_. 

    ------------------------------------------------------------
   
3. C3 - C2 $d_{j} = 1.0$ || C7 - C5 $d_{j} = 0.5$ || C9 - C39 = $d_{j} = 1.0$:

    * In the *C3 - C2* match **B** links two mentions to 'JHWH'. The JHWH class occurs in the *C48* - *C44* match. **B**'s separate *C2* is probably caused by a linking error, since there is no reason to not link 'JHWH' in this class to his greater JHWH class (*C44*), the three mentions are counted as 6x _Annotator Error_. 
        Where **A** distinguishes a seperate JHWH class in his *C48*, **B** seems to distinguish multiple JHWH classes: *C47*, *C37*, *C14*, *C39*, *C2* and *C44*. **B** seems to have made either linking errors or a conscious choice to seperate the references to JHWH. Whether all **A**'s mentions in his JHWH class are correct is debatable, the mentions in **B**'s *C39* are however clear cases of JHWH references, and are thus not linked correctly: 2x _Annotator Error_.
    * The differences between **A**'s and **B**'s annotations in *C3* - *C2* further focus on the understanding of the subject NP's in 10:4 'NFJ>JM', 'R>CJ >LPJ JFR>L': 'the leaders', 'the heads of the thousands of Israel'. **B**'s *C5* is **A**'s *C3* and *C7* minus the infinitive construct 'HQHJL' combined. 
    * **B**'s'HQHJL' ('to assemble') in 10:7 in his *C5* is an infinitive construct which **A** has corefered with 'QHL' (substantive, 'assembly') in his *C9* because of their lexical identity. **B** has annotated 'QHL' with the article 'HQHL' -- the annotation guidelines do not allow for an annotation of the article -- and adds the mention to his singletons. These are counted as 4x _Annotator Error_. 
    * **B**'s *C5* interprets the subject NP's 'NFJ>JM' and 'R>CJ >LPJ JFR>L' (appositive) as blowing (2x 'TQ<TM', verb, 2Mpl) the trumpets, which seems to be incorrect since the 2Mpl is a general instruction by YHWH as to how the assembly of Israel should be gathered. The two NP's are subject of the VP 'NW<DW' (verb, 3Cpl, 'to appoint'). The 2Mplurals of 'TQ<TM', 'TQ<TM', 'TTQ<W' (verb, 2Mpl, 'to blow') and 'TRJ<W' (verb, 2Mpl, 'to shout') therefore do not have an explicit subject. Except for their occurrence in 10:4, nowhere else the leaders of Israel are named as explicit subject. **A**'s *C3* and *C7* therefore seem to be a correct interpretation of the referents. The differences in annotations are counted and categorised as 7x _Referents_ (3x for the subject NP's and 4x for the verbs). 

    ------------------------------------------------------------

5. C5 - C6 $d_{j} = 0.5$ || C6 - C7 $d_{j} = 0.4$: 

    * The difference in annotations between **A**'s and **B**'s *C5* - *C6* (in Nu 10:5) and *C6 - C7* (Nu 10:6) comes down to the adverbial phrases (AdvP) that should be annotated with XNJM (verb, participle, Mpl, 'to encamp') or not: 2x 'XNJM' (**A**, *C5* and *C6*) or 'HXNJM QDMH' and 'XNJM TJMNH' (**B**,  *C6* and *C7*). 
    * The annotation guidelines allow for the annotation of adverbial phrases, and in this case the AdvP's that **B** annotated distinguishes the two different camps from each other: 'XNJM QDMH' are the east camps, 'XNJM TJMNH' are the south camps. **A** has made 2x _Annotator Error_. **B** has erroneously annotated the article 'H', this is also 1x _Annotator Error_. 3x _Annotator Error_ in total.

    ------------------------------------------------------------

6. C8 - C47 $d_{j} = 1.0$ || C48 - C44 $d_{j} \approx 0.6957$: 

    * The *C8* - *C47* match is a total disagreement between **A** and **B**. 
    * Where **A** sees a class (*C8*) in 2x undetermined object NP 'TRW<H' (substantive, 'shouting/blast') in 10:5 and 10:6, **B** adds ('TRW<H', 6) and ('TRW<H', 8) to his singletons. Though the substantive is undetermined it seems to be the same kind of blast that signals the camp for setting out. They can therefore be corefered: 4x _Referents_. 2x for **A** and 2x for **B**.
   * As was the case with the *C3 - C2* match, **B**'s *C47* ('CWBH', 'JHWH'), which is a JHWH class, should have been linked to his *C44*. 4x _Annotator Error_.

    ------------------------------------------------------------

7. C14 - C16 $d_{j} = 0.6$ || C35 - C15 $d_{j} = 1.0$:
 
    * **B** annotates  two separate classes that refer to a 'cloud' (*C16*, *C15*), where **A** annotates one class: *C14* and specifically links the references of clouds to the '<NN JHWH', 'cloud of JHWH'. **B** adds this mention to his singletons. The question is if **B** is right in annotating these classes seperately or not, or conversely: if **A** is right in annotating one class or not. 
    * The first occurence of '<NN' in 10:11 is a determined subject NP with 3Msg verb/VP 'N<LH' ('to ascend'). The cloud ascends from the tabernacle. Suggesting it is the cloud of God. 'JCKN' and '<NN' occur in 10:12, '<NN' is a determined subject NP, 'JCKN' 3Msg verb/VP ('to dwell'). '<NN JHWH' occurs in 10:34 and is a determined subject NP. The subject NP in 10:34 seems to identify the previous mentions definitely. Here it is also not clear whether **B** made a linking error, or sees his two classes as two different entities. We count 6x _Genuine Ambiguity_: 3x for the *C14* - *C16* match, 2x for the *C35* - *C15* match, and 1x for **B**'s singleton. 
    * Then **A**'s *C35*. **A** links one occurrence of 'YB>T' (armies) to 'JS<W' for *C35* in 10:28. The rest of the singular ('YB>') and plural occurrences are added by **A** to his singletons. **B** adds all mentions of 'YB>' to his singletons. There are 5x 'YB>T' and 4x 'YB>' mentions in the absolute state in Nu 10. 'JS<W' takes as implicit subject 'YB>T'. **B** has missed this class: 2x _Annotator Error_. There is disagreement between **A** and **B** on the annotation of the other mentions of 'army'. Where **A** adds the other 4 mentions of 'YB>T' to his singletons, **B** finds that there are 0 mentions of 'YB>T'. This is a result of **B**'s erroneous annotations of the suffix 'TM' (should be 'M') which has been discussed already. That means that these errors are counted as 4x _Annotator Error_. **A** annotates 4x 'YB>', where **B** annotates 9x 'YB>'. The difference between **A**'s and **B**'s annotations of 'YB>' is 5. These 5 are all the mentions that **B** has not annotated correctly: 5x _Annotator Error_.
    
    ------------------------------------------------------------
    
8. C10 - C8 $d_{j} = 0.5$ || C11 - C12 $d_{j} = 0.45$ || C45 - C1 $d_{j} \approx 0.6316$: 

    * **B**'s diff indicates the disagreement between **A** and **B** in the *C10* - *C8* match. **A** links ('HR<TM', 3), ('NZKRTM', 4), ('KM', 5) to his *C11*. 
    * The disagreement focusses on whether the 2Mpl of the three mentions refer to the 'sons of Aaron', 'the priests' ('BNJ >HRN', 'KHNJM'). **A** says they all refer to a bigger 2Mpl group (*C11*). **B** takes the context of 10:8 in consideration where it is stated that only the sons of Aaron are to blow the trumpets. 'HR<TM' (verb, 2Mpl, 'to blow/shout') then could definitely refer to the priests, whereas the referent of 'NZKRTM' (verb, 2Mpl, 'to remember') and subsequent 'KM' is debatable. Does the 2Mpl only refer to the priests (**B**'s *C8*), or to the people of Israel, (the 2Mpl of **A**'s *C11*). Both interpretations are possible. Therefore the disagreement is counted as 6x _Referents_ (3x for the disagreement in *C10* - *C8* and 3x for  *C11* - *C12*). 
    * The disagreement between **A** and **B** *C11* - *C12* is a result of the difference in *C10* - *C8*, 5x 'KM' (the sixth 'KM' comes from *C10* - *C8*) and 'TB>W' (verb, 2Mpl, 'to come'). **B** links 'TB>W' and 4x 'KM' to his *C1* (cf. **B**'s diff); the 5th 'KM' was added to his *C36*, and will be accounted for below. The four suffixes 'KM' and 'TB>W' that **B** has linked to his *C1* are strange, since he has also linked Moses 1x 'MCH' to this class, but also to 1x 'MCH' *C36* and *C43*. We leave 'MCH' for what it is, since there is no good explanation for this linking. The 4 missing 'KM' and 'TB>W' in **B**'s *C12* are categorised as 5x _Annotator Error_ since they seem to refer to the general 2Mpl and not to Moses. 
    * **A**'s *C45* is a Moses 'MCH' class, **B** links 1x MCH to his *C1*. **B** misses 4x 'J>MR' (verb, 3Msg, 'to speak'). The 4x 'J>MR' refer to Moses: 4x _Annotator Error_. The 4 added 'KM' and 'TB>W' in **B**'s *C1* are categorised as 5x _Annotator Error_ since they seem to refer to the general 2Mpl and not to Moses. The 'W' that **A** has annotated but **B** not, is hard to trace in **B**'s annotations. It is therefore classified as 1x _Referents_. 
    
    ------------------------------------------------------------ 
   
9. C12 - C10 $d_{j} \approx 0.3333$: 
    
    * **A** adds '>JBJ' (substantive, 'enemy') to his singletons, whereas **B** adds the mention to his *C10*. 'YR' also means 'hostile' or 'enemy'. Both **A**'s and **B**'s linking choices are thus possible. The difference is counted as 2x _Annotator Error_ (1x for this match, 1x for the singletons).     
    
    ------------------------------------------------------------

10. C15 - C17 $d_{j} = 0.6$ || C16 - C18 $d_{j} \approx 0.3333$: 

    * The differences between **A**'s and **B**'s annotations in these matchings relate to the suffixes 'TM'/'M' (3Mpl) and 'W' (3Msg). 
    * 'TM' is an erroneous annotation. The 'Taw' belongs to the plural form of 'YB>T' ('army'), not to the suffix. 2x _Annotator Error_. 
    * To whom the 'W' 3Msg  refers is a question of how the relation between the linguistic elements should be understood. **A** says it refers the subject NP's 'NXCWN', 'BN&<MJNDB' ('Nahshon, son of Amminadab'), **B** says it refers back to the subject NP 'DGL MXNH BNJ&JHWDH' ('division of the camp of the sons of Judah'). The 3Mpl within the same wayyiqtolX (X=explicit subject) clause refers to the subject NP which is also plural; whereas in the ensuing nominal clause the 3Msg refers to the 3Msg 'Nahshon, son of Amminadab' and not back again to the plural 'divisions of the camp of Judah'. **B**'s 'W' seems to be 1x _Annotator Error_.

    ------------------------------------------------------------

11. C19 - C21 $d_{j} = 0.25$: 

    * **A** links the determined 'MCKN H<DT' to his *C19*; **B** adds this mentions to his singletons. 'MCKN H<DT' is the same dwelling place of God as the other occurences of 'MCKN'. Therefore: 2x _Annotator Error_.

    ------------------------------------------------------------
    
12. C21 - C22 $d_{j} = 0.6$ || C22 - C24 $d_{j} \approx 0.3333$: 

    * Like under point 8 **A**'s and **B**'s annotations in these matchings relate to the suffixes 'TM'/'M' (3Mpl) and 'W' (3Msg). 
    * 'TM' is an erroneous annotation. The 'Taw' belongs to the plural form of 'YB>T' ('army'), not to the suffix. 2x _Annotator Error_.
    * To whom the 'W' 3Msg  refers is a question of how the relation between the linguistic elements should be understood. **A** says it refers the subject NP's '>LJYWR', 'BN&CDJ>WR' ('Elizur, son of Shedeur'), **B** says it refers back to the subject NP 'DGL MXNH R>WBN' ('division of the camp of Rueben'). The 3Mpl within the same weqatalX (X=explicit subject) clause refers to the subject NP; whereas in the ensuing nominal clause the 3Msg refers to the 3Msg 'Elizur, son of Shedeur' who is commander over *his* army, and not back again to the 'division of the camp of Rueben'. **B**'s 'W' seems to be 1x _Annotator Error_.

    ------------------------------------------------------------
    
13. C25	- C27 $d_{j} = 0.5$:

    * The disagreement between **A** and **B** in 10:21 is a result of a different understanding of who the subject of 'HQJMW' (verb, 3Cpl, 'to arise/set up') is and of an annotation error. 
    * The question is if 'HQJMW' refers to the Kohathites as 'carriers of holy things' (`NF>J HMQDC`), or refers to a general 3Cpl: before they arrived (`B>M`, infc, predicate subject, 3Mpl), they [others] had set up the tabernacle.' **A** adds the verb to his singletons, **B** to his *C27*. We count 2x _Genuine Ambiguity_ (1x for **A**'s singleton and 1x for **B**'s' HQJMW'.)
    * **B**'s suffix 3Mpl is an error, it goes with his singleton: (`B>`, 41). The suffix is part of `B>M` (like in **A**'s *C25* ), infc, predicate subject, 3Mpl, which should be annotated as one mention: 2x _Annotator Error_.

    ------------------------------------------------------------

14. C26 - C28 $d_{j} = 0.6$ || C27 - C29 $d_{j} \approx 0.3333$: 

    * Like under point 8 an 10 **A**'s and **B**'s annotations in these matchings relate to the suffixes 'TM'/'M' (3Mpl) and 'W' (3Msg). The annotations occur in 10:22. 
    * 'TM' is an erroneous annotation. The 'Taw' belongs to the plural form of 'YB>T' ('army'), not to the suffix. 2x _Annotator Error_.
    * To whom the 'W' 3Msg  refers is again a question of how the relation between the linguistic elements should be understood. **A** says in his *C27* that it refers to the subject NP's '>LJCM<', 'BN&<MJHWD' ('Elishama son of Ammihud'). **B** says it refers back to the subject NP 'DGL MXNH BNJ&>PRJM' ('division of the camp of the sons of Ephraim'). The 3Mpl within the same weqatalX (X=explicit subject) clause refers to the subject NP; whereas in the ensuing nominal clause the 3Msg refers to the 3Msg 'Elishama son of Ammihud' who is commander over *his* army, and not back again to the camp of Ephraim. **B**'s 'W' seems to be 1x _Annotator Error_.

    ------------------------------------------------------------    
 
15. C30 - C32 $d_{j} = 0.4$ || C31 - C33 $d_{j} \approx 0.3333$: 

    * This is the same annotation pattern as discussed under points 8, 10 and 12. **A** annotates suffix 'M' 3Mpl in his *C30*; **B** annotates suffix 'W' 3Msg in his *C32*. In the *C31* -- *C33* match **A** links the suffix to his *C31*. The annotations occur in 10:25. 
    * Like with the other references to the divisions of the tribes, the suffix 'M' 3Mpl refers back to subject NP 'DGL MXNH BNJ&DN' ('division of the camp of the sons of Dan'). The suffix 'W' 3Msg refers to the commander of the division. **B** made 2x _Annotator Error_, 1x for 'M' and 1x 'W'. 

    ------------------------------------------------------------ 

16. C34 - C37 $d_{j} = 1.0$:
    
    * **A**'s *C34* relates to **B**'s singletons. 
    * In 10:28 **B** adds '>LH' (subject DP, 'these') and 'MS<J BNJ&JFR>L' (NP, 'order of the sons of Israel') to his singletons. The DP refers to the NP. For **B** 4x _Annotator Error_ is counted. 
    * The suffix 'M' 3Mpl, is probably linked by **B** to his *C36* which is either ('TM', 4) or ('TM', 5) in that class. 3Mpl clearly refers to 'MS<J BNJ&JFR>L' and 'TM' is an annotator mistake: 2x _Annotator Error_.
    * **B**'s annotations in his *C37* (('>MR', 0), ('JHWH', 1), ('>TN', 2)) should have been linked to his JHWH class (*C44*), since there is an explicit reference to JHWH in it. 6x _Annotator Error_.
    
    ------------------------------------------------------------    

17. C36 - C43 $d_{j} \approx 0.1579$: 

    * The difference between **A**'s and **B**'s is a result of **B**'s diff [('J>MR', 16), ('MCH', 17), ('J>MR', 18)]. Both **A** and **B** identify their classes as Hobab, son of Reuel. the Midianite, father-in-law of Moses ('XBB', `BN&R<W>L`, 'MDJNJ', 'XTN MCH'). **B**'s linking of 'MCH', 'Moses', and 2x verb 'J>MR' (3Msg, 'to speak') is not possbible. The class is either Moses or his father in law Hobab, not both. Since **B** has already identified Moses in his *C1*, **B**'s diff is counted as 3x _Annotator Error_.

    ------------------------------------------------------------ 
    
18. C39 - C36 $d_{j} \approx 0.7083$ || C42 - C42 $d_{j} \approx 0.6364$ || C37 - C14 $d_{j} = 1.0$:  

    * The annotations in matches *C39* - *C36* and *C42* - *C42* stand in relation to each other. **A** annotates in his *C39* a 1Csg group which he does not identify, i.e. he does not link it with an explicit NP etc. **B** also annotates an 1Csg group but identifies it with the subject NP 'BNJ&JFR>L' ('tribes of Israel') from 10:12. 
    * The disagreement thus focusses on the identification of the 'we'-group that Moses speaks about in 10:29--31. It is possible to link the 1Csg group to Israel, since it is identified later on in 10:29. With the identification of the 1Csg group the question comes up if part-of relations should be annotated as well. Strictly speaking Moses also belongs to the we-group along with people that are possibly unidentified in the text. If Moses were linked to the we-group there would originate one big we-group which does not do justice to the text. Because of these linking possibilities the disagreement is counted as ambiguity and calculated as the symmetric difference of **B** (16x) minus **A**'s difference in his *C37* (2x) minus **B**'s second erroneously annotated suffix 'TM' (1x, for the other 'TM' see point 14). **A** has erroneously not linked ['NS<JM', '>NXNW'], '>NXNW' is an IPP 1Cpl, to the we-group in his *C39*: therefore 13 x _Genuine Ambiguity_ is counted. 'NS<JM' and '>NXNW' are counted as 2x _Annotator Error_. That leaves **A**'s diff in the *C39* - *C36* match. ('XNTNW', 3) in 10:31 is an infinitive, predicate subject with suffix 1Cpl ('to encamp') and should be annotated as one mention. **B** adds 'XNT' to his singletons. 2x _Annotator Error_.
    * **B** annotates a class (*C44*) that refers to 'PNJ JHWH', 'face of YHWH', but has also added 'JHWH' and '>LHJ' to this class. **A** annotates one class of references to JHWH, also the occurences of 'PNJ JHWH'. In the context of this text 'PNJ JHWH' could be seen as a reference to JHWH as a whole. There are two explanations for **B**'s choice to split the references to JHWH in a 'JHWH only' class and a 'PNJ JHWH' class. The first explanation is that **B** sees the face of God as a different entity than God himself. The question then is why **B** also linked ('JHWH', 4) and ('>LHJ', 5) to his *C14*, since these mentions are explicit and clear references to JHWH, which is **B**'s *C44*. The second explanation is that **B** made a linking error in brat. It is not clear how these annotation differences should be interpreted. They are therefore counted and categorised as 6x _Referents_, 1x for each difference in mentions, except 'MPNJ' in 10:35 which is an annotation mistake (the M- is a preposition): 1x _Annotator Error_. 
    * **B** adds J>MR' and 'MCH' to his *C36*. This linking to his class which he identifies as 'BNJ&JFR>L', can only be explained as 2x _Annotator Error_. **B**'s *C1* seems to be most clearly his 'MCH' class (**B** links 'MCH' also to his *C1*, *C36* and *C43*). 
    * The annotation choices that **A** and **B** have made in *C37* - *C14* and *C39* - *C36* are visible in *C42* - *C42*. **A** sees one class 'BNJ&JFR>L' and links 'JS<W', 'HM', 'HM', 'HM' to his *C42*, in contrast to **B** who sees a seperate class in these mentions and adds an extra 'M'. It is not clear why **B** considers these mentions to be a seperate class. The difference between **A** and **B** is counted as 4x _Genuine Ambiguity_. **B**'s 'M' is an annotation error which should have been 'HM': 1x _Annotator Error_.

    ------------------------------------------------------------ 

19. C40 - C40 $d_{j} \approx 0.3333$: 

    * **A**'s 'VWB' is added by **B** to his singletons. 
    * In 10:32 'VWB'and 'HW>' occur in one determined subject NP. The translation of the subject NP would be: the good, he [is]. As such they VWB should be corefered, it is the same entity. 2x _Annotator Error_ (1x singletons, 1x for the diff in this class).

    ------------------------------------------------------------ 
  
20. C43 - C9 $d_{j} = 1.0$: 

    * **A** distinguishes for *C43* 2x 'DRK CLCT JMJM' in 10:33. Both mentions refer to the same time slot: the three days that YHWH and the ark travel. **B** erroneously adds these mentions to his singletons: 4x _Annotator Error_ (2x for the class difference and 2x for the singleton difference).
    * **B** distinguishes an entity in his *C9* with ['HJW', 'XQT <WLM'] in 10:8. 'HJW' is a verb, 3Cpl, 'to be' and 'XQT <WLM' is an NP 'regulation of eternity' with function predicate complement and can thus not be subject of 'HJW'. It is however possible to corefer these mentions since the verb seems to refer to the 'regulation of eternity': '[these]\they are the regulation of eternity'. The difference between **A** and **B** is categorised as 4x  _Referents_. 2x for the difference in this maching, and 2x for the difference in singletons. 
    
    (C43 - C4 $d_{j} = 1.0$:)
    * The *C43* - *C4* match is discussed under point 1. **A** however has added one occurrence of 'MXNWT' of this singletons. This difference is counted as 1x _Annotator Error_. 
    
    ------------------------------------------------------------

21. C45 - C1 $d_{j} \approx 0.6316$: 

    * As has been discussed above, **B** links a number of mentions to his *C1* 'Moses' class, which should not have been corefered with Moses. **B**'s diff ('KM', 7), ('KM', 8), ('TB>W', 9), ('KM', 10), ('KM', 11) is counted as 5x _Annotator Error_. 

    ------------------------------------------------------------

22. C48 - C44 $d_{j} \approx 0.6957$: 

    * As has been discussed above, **B** does not link a number of mentions to his *C44* 'JHWH' class which should have been linked to it. The 16 mentions that **B** misses are counted as 16x _Annotator Error_. 

    ------------------------------------------------------------

23. S - S $d_{j} \approx 0.4651$: 
    
    * Most of the differences in annotation of singletons have been discussed. There are however some interesting differences left for discussion. First **A**'a diff will be discussed then **B**'s diff. 
    * **A**'s ('LMS<', 2) is an erroneous annotation. This should be: 'MS<'. The same goes for ('B> MVH BNJ >CR', 52), this should be 'YB> MVH BNJ >CR' as **B** has annotated (index 50). 2x _Annotator Error_. 'JWMM' is a time indication that can be annotated. It can be classified as 1x _Generics_. 
    * **B**'s 'HQHL' and 'TWR' are both an infc without predicate subject or subject, and should not be annotated: 2x _Annotator Error_. 'NS<' ('pull out') in 10:35 is also an infc without suffix, there more occurrences of NS<. An infc can be annotated if it has a lexical identity with the same mention elsewhere, the guidelines are not clear on if this mention should also be corefered: 1x _Guidelines_. 

#### Conclusion Numbers 10

The count and categorisation of the differences in annotations between **A** and **B** for Numbers 10 are:

* 36x _Referents_
* 141x _Annotator Error_
* 25x _Genuine Ambiguity_
* 1x _Generics_
* 1x _Guidelines_

Most differences are clearly a result of _Annotator Error_ 's: linking errors, erroneous mention annotation, corefering infinitives etc. Some of these mistakes can be prevented by a more clear annotation process, and a different presentation of the data that should be annotated.

### 5.3 Numbers_009.iaa

In [38]:
nu009_df = MakeTable('iaa-files/Numbers_009.iaa')

nu009_df

,ann_A,ann_B,L,M,R,D,d
0,C1,C4,2,4,0,2,0.3333
1,C2,C28,2,0,9,11,1.0000
2,C3,C3,6,11,0,6,0.3529
3,C4,C27,2,0,2,4,1.0000
4,C5,C5,0,2,0,0,0.0000
5,C6,C9,5,8,0,5,0.3846
6,C7,C26,4,0,2,6,1.0000
7,C8,C1,1,1,4,5,0.8333
8,C9,C20,3,0,2,5,1.0000
9,C10,C10,4,5,1,5,0.5000


In [39]:
nu009_df.loc[(nu009_df['d'] == 0)]

,ann_A,ann_B,L,M,R,D,d
4,C5,C5,0,2,0,0,0.0
14,C15,C15,0,2,0,0,0.0
18,C19,C18,0,2,0,0,0.0
19,C20,C19,0,5,0,0,0.0
20,C21,C21,0,2,0,0,0.0
22,C23,C22,0,3,0,0,0.0
23,C24,C23,0,2,0,0,0.0
24,C25,C24,0,7,0,0,0.0
25,C26,C2,0,2,0,0,0.0


In [40]:
nu009_df.loc[(nu009_df['d'] >= .14)]

,ann_A,ann_B,L,M,R,D,d
0,C1,C4,2,4,0,2,0.3333
1,C2,C28,2,0,9,11,1.0000
2,C3,C3,6,11,0,6,0.3529
3,C4,C27,2,0,2,4,1.0000
5,C6,C9,5,8,0,5,0.3846
6,C7,C26,4,0,2,6,1.0000
7,C8,C1,1,1,4,5,0.8333
8,C9,C20,3,0,2,5,1.0000
9,C10,C10,4,5,1,5,0.5000
10,C11,C11,6,4,0,6,0.6000


In [ ]:
retrieve_ann('chris_A/Numbers_009.ann', 'gyus_B/Numbers_009.ann')

## 6. Overview  Categorisation

Below is given an overview of the results of the IAA analysis of Ps 67, 101, 17, 20, 32, 70 and Nu 8 and 10. 

In [54]:
cats = ['Annotator Error',
        'Genuine Ambiguity', 
        'Generics', 
        'Guidelines', 
        'Referents', 
        'NP', 
        'Suffix', 
        'Verb', 
        'IPP',
        'Named Entity', 
        'DP', 
        'Appositive', 
        'brat']

Data = {'Category': cats,
        'Count': [533, 48, 1, 11, 51, 2, 0, 0, 0, 0, 0, 4, 2]
       }

cat_df = pd.DataFrame(Data, columns = ['Category', 'Count']).sort_values(
                            by=['Count'], ascending=False)
cat_df

,Category,Count
0,Annotator Error,533
4,Referents,51
1,Genuine Ambiguity,48
3,Guidelines,11
11,Appositive,4
5,NP,2
12,brat,2
2,Generics,1
6,Suffix,0
7,Verb,0


In [57]:
#print(cat_df.to_latex(index=False))
ExportToLatex(OUTPUT, 'cat_iaa_analysis', cat_df, indx = False)

